# 📌 Projeto: Case Prático - Banco Shield - Tratamento de Dados

**Autora:** **Daniele Cardoso**  
**Objetivo do notebook:** realizar o tratamento, padronização e validação de qualidade da base **fato_contratos**, assegurando consistência mínima para análises, modelagem dimensional e construção de dashboards (visão executiva e indicadores operacionais/financeiros).

---
:
## 🎯 Contexto do Case
O case envolve duas instituições (**Banco Shield** e **Hidra**) e um cenário de **baixa confiabilidade dos dados**, com inconsistências que comprometem análises e decisões (ex.: valores ausentes, formatação irregular, chaves inconsistentes e regras de domínio violadas).  
Este notebook aplica um pipeline de limpeza e checagens para elevar a qualidade dos dados e gerar uma versão final **Gold** do dataset.

---

## ✅ Escopo do Pipeline (O que este notebook entrega)
Ao final do processo, o notebook entrega:

- **Dataset tratado (Gold)** pronto para consumo em BI/SQL/Modelagem.
- **Relatórios de auditoria e validação** por etapa (logs em CSV), permitindo rastrear:
  - Linhas removidas e motivos;
  - Campos corrigidos/normalizados;
  - Imputações realizadas (com contagem e evidências);
  - Taxas de inconsistência antes/depois.
- **Validações finais** de conformidade com regras mínimas de dados, incluindo:
  - Presença de colunas obrigatórias;
  - Tipagem coerente (numéricos convertíveis e campos essenciais completos);
  - Domínio de valores (não-negatividade e faixas válidas);
  - Integridade referencial com dimensões (`dim_produto` e `dim_localidade`);
  - Checagem de duplicidades com chave “soft” definida.

---

## 🧪 Principais Regras de Qualidade Aplicadas
Este notebook adota regras **hard** (falha implica correção/remoção) e regras **soft** (sinalização como alerta quando aplicável), priorizando consistência para análise sem mascarar comportamentos reais do negócio.

Exemplos de validações:
- **Valores numéricos** convertíveis (`financed_amount`, `outstanding_balance`) e **não negativos**;
- **`ano_mes`** padronizado no formato **YYYYMM** (6 dígitos; mês entre 01 e 12);
- **Chaves de dimensão** válidas:
  - `product_id` deve existir na `dim_produto`;
  - `location_id` deve existir na `dim_localidade`;
- **Eliminação de registros** inviáveis para modelagem dimensional (ex.: `product_id` ausente).

---

## 📦 Saídas e Artefatos Gerados
Os principais outputs gerados ao longo do notebook incluem:

- **Versões intermediárias** do dataset tratado (`clean_vX.csv`) para rastreabilidade;
- **Logs de auditoria** por etapa (ex.: `cleaning_audit_stepX.csv`);
- **Relatório de validação final** (resumo PASS/FAIL e evidências quando necessário);
- **Versão final Gold** do CSV para download e carga em banco/dashboards.

---

## 🔍 Como ler este notebook
O pipeline está organizado em **passos numerados**, com padrão consistente:
1. Carregamento e padronização inicial  
2. Regras de limpeza e correções (NaN, tipo, domínio)  
3. Validações e integridade referencial  
4. Auditoria comparativa (antes vs. depois)  
5. Export final (Gold) e logs para rastreabilidade

---

## 📌 Observação importante
O foco do processo é **garantir confiabilidade** e **explicar as decisões de tratamento**, evitando correções “silenciosas”.  
Toda alteração relevante é registrada em logs para permitir auditoria e justificar impactos no resultado final.

---


## PASSO 0 — Importação de todas as bibliotecas necessárias para o tratamento dos dados, auditoria e manipulação dos arquivos

In [1]:
from pathlib import Path
import pandas as pd
import re
import os
import glob
import numpy as np

## PASSO 1 — Configuração do projeto, estrutura de pastas e leitura dos dados

**Objetivo deste passo**  
Padronizar o ambiente do notebook para garantir **reprodutibilidade**, **organização** e **portabilidade** (rodar no Colab com Drive ou em execução local). Este bloco cria a estrutura padrão do projeto e carrega as tabelas necessárias para o tratamento e validações.

**O que será feito**
1. **Montar a estrutura de diretórios** do projeto:
   - `data/raw`: dados brutos (originais, sem alterações)
   - `data/processed`: dados tratados (saídas do pipeline)
   - `quality_logs`: evidências e relatórios de qualidade/auditoria
2. **Centralizar caminhos** em variáveis (`PROJECT_ROOT`, `RAW_DIR`, `PROCESSED_DIR`, `LOG_DIR`) para evitar “hardcode” espalhado pelo notebook.
3. **Validar se os arquivos existem** antes de tentar carregar (evita erros silenciosos e facilita o diagnóstico).
4. **Carregar os datasets**:
   - `fato_contratos.csv`
   - `dim_produto.csv`
   - `dim_localidade.csv`
5. Gerar uma **foto rápida inicial** (shapes, valores únicos de `bank`, `min/max` de `ano_mes`) para registrar o estado do dado **antes** da limpeza.

**Por que isso importa**
- Um avaliador consegue rodar seu notebook do zero sem “adivinhar” caminhos e configurações.
- A separação `raw/processed/logs` facilita auditoria e sustentação da entrega no GitHub.
- O “baseline” (shapes e domínios) dá contexto imediato do problema (ex.: variações de `bank`, `ano_mes` inválido).

In [3]:
# =========================================================
# PASSO 1 — CONFIGURAÇÃO DO PROJETO (portabilidade + padrão)
# =========================================================

# (A) Se você estiver usando Google Drive, descomente as 2 linhas abaixo:
# from google.colab import drive
# drive.mount("/content/drive")

# (B) Defina a raiz do projeto:
# - Se estiver no Drive, use algo como:
#   PROJECT_ROOT = Path("/content/drive/MyDrive/Case/case-shield-hidra")
# - Se quiser rodar sem Drive (upload local), use:
#   PROJECT_ROOT = Path("/content/case-shield-hidra")

PROJECT_ROOT = Path("/content/drive/MyDrive/Case/case-shield-hidra")  # <-- ajuste aqui

RAW_DIR       = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
LOG_DIR       = PROJECT_ROOT / "quality_logs"

for p in [RAW_DIR, PROCESSED_DIR, LOG_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("✅ PROJECT_ROOT:", PROJECT_ROOT)
print("✅ RAW_DIR:", RAW_DIR)
print("✅ PROCESSED_DIR:", PROCESSED_DIR)
print("✅ LOG_DIR:", LOG_DIR)

# (C) Arquivos esperados
files_expected = {
    "fato_contratos": RAW_DIR / "fato_contratos.csv",
    "dim_produto": RAW_DIR / "dim_produto.csv",
    "dim_localidade": RAW_DIR / "dim_localidade.csv",
}

print("\n--- Checagem de arquivos ---")
for name, path in files_expected.items():
    print(f"{name}: {'OK' if path.exists() else 'NÃO ENCONTRADO'} -> {path}")

# (D) Se todos existirem, carregue e mostre um resumo rápido
if all(p.exists() for p in files_expected.values()):
    df_contratos = pd.read_csv(files_expected["fato_contratos"], low_memory=False)
    df_produto = pd.read_csv(files_expected["dim_produto"], low_memory=False)
    df_localidade = pd.read_csv(files_expected["dim_localidade"], low_memory=False)

    print("\n✅ Shapes:")
    print("df_contratos  :", df_contratos.shape)
    print("df_produto    :", df_produto.shape)
    print("df_localidade :", df_localidade.shape)

    print("\n✅ Banks únicos:", sorted(df_contratos["bank"].dropna().unique().tolist()) if "bank" in df_contratos.columns else "coluna bank não encontrada")
    print("✅ ano_mes min/max:", (df_contratos["ano_mes"].min(), df_contratos["ano_mes"].max()) if "ano_mes" in df_contratos.columns else "coluna ano_mes não encontrada")
else:
    print("\n⚠️ Coloque os CSVs na pasta data/raw com os nomes exatos acima e rode novamente.")


✅ PROJECT_ROOT: /content/drive/MyDrive/Case/case-shield-hidra
✅ RAW_DIR: /content/drive/MyDrive/Case/case-shield-hidra/data/raw
✅ PROCESSED_DIR: /content/drive/MyDrive/Case/case-shield-hidra/data/processed
✅ LOG_DIR: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs

--- Checagem de arquivos ---
fato_contratos: OK -> /content/drive/MyDrive/Case/case-shield-hidra/data/raw/fato_contratos.csv
dim_produto: OK -> /content/drive/MyDrive/Case/case-shield-hidra/data/raw/dim_produto.csv
dim_localidade: OK -> /content/drive/MyDrive/Case/case-shield-hidra/data/raw/dim_localidade.csv

✅ Shapes:
df_contratos  : (6000, 11)
df_produto    : (20, 5)
df_localidade : (15, 4)

✅ Banks únicos: ['Banco S.H.I.E.L.D', 'Banco Shield', 'HIDRA ', 'Hidra', 'Hydra']
✅ ano_mes min/max: (202501, 202513)


## PASSO 2 — Auditoria pré-limpeza (baseline de qualidade + evidências)

**Objetivo deste passo**  
Gerar uma auditoria de qualidade **ANTES** de qualquer tratamento. Essa auditoria serve como *baseline* e cria **evidências objetivas** para justificar todas as alterações posteriores (correções, remoções e padronizações).

**O que será verificado**
1. **Domínio de `bank`**: listar variações existentes para orientar a normalização.
2. **Validade de `ano_mes`**: identificar valores fora do intervalo esperado e meses inválidos (ex.: `202513`).
3. **Integridade referencial (FKs)**:
   - `product_id` da fato deve existir na `dim_produto`
   - `location_id` da fato deve existir na `dim_localidade`
4. **Unicidade de `contract_id`**: detectar duplicidades.
5. **Regras numéricas e lógicas** (se as colunas existirem):
   - `risk_score` deve ficar entre **0 e 1**
   - valores negativos/NaN em campos que não devem permitir isso
   - regra de inadimplência: **`dpd < 30` não pode ter `delinquent_amount_30p > 0`**

**Por que isso importa**
- O avaliador consegue ver, com números, *quais problemas existiam* no dado bruto.
- Você consegue sustentar tecnicamente cada decisão do tratamento (por regra violada).
- Esse relatório vira evidência direta no GitHub (`quality_profile_pre_v1.csv`) e material da apresentação (slide de “Diagnóstico Inicial”).

**Saída esperada**
- Uma tabela consolidada (`audit_df`) com:
  - nome do check
  - quantidade de linhas inválidas
  - percentual de falha
  - detalhes do achado
- Salvamento do relatório em: `quality_logs/quality_profile_pre_v1.csv`

In [4]:
# =========================================================
# PASSO 2 — AUDITORIA PRÉ-LIMPEZA (baseline de qualidade)
# =========================================================

def add_check(results, check_name, bad_count, total_count, details=None):
    results.append({
        "check": check_name,
        "bad_count": int(bad_count),
        "total_count": int(total_count),
        "bad_pct": float(bad_count) / float(total_count) if total_count else 0.0,
        "details": details
    })

total_rows = len(df_contratos)
audit = []

# ---------------------------
# 1) Domínio / padronização de BANK
# ---------------------------
if "bank" in df_contratos.columns:
    banks = df_contratos["bank"].astype(str).fillna("").str.strip()
    unique_banks = sorted(banks[banks != ""].unique().tolist())
    add_check(
        audit,
        "bank_domain_variants (apenas informativo)",
        bad_count=max(len(unique_banks) - 2, 0),  # heurística: ideal 2 bancos
        total_count=total_rows,
        details=f"Valores únicos encontrados: {unique_banks}"
    )
    print("✅ Valores únicos de bank:", unique_banks)
else:
    add_check(audit, "bank_column_missing", bad_count=total_rows, total_count=total_rows, details="Coluna bank ausente")

# ---------------------------
# 2) Validação de ano_mes (YYYYMM e mês 01-12)
# ---------------------------
if "ano_mes" in df_contratos.columns:
    ano_mes = pd.to_numeric(df_contratos["ano_mes"], errors="coerce")
    # inválidos: nulos, não numéricos, ou mês fora 01-12
    month = (ano_mes % 100)
    invalid_format = ano_mes.isna()
    invalid_month = (~invalid_format) & ((month < 1) | (month > 12))

    # aqui você pode ajustar o range esperado se necessário
    min_expected = 202501
    max_expected = 202512
    out_of_range = (~invalid_format) & ((ano_mes < min_expected) | (ano_mes > max_expected))

    add_check(audit, "ano_mes_invalid_format (NaN/non-numeric)", invalid_format.sum(), total_rows,
              details="Valores que não puderam ser convertidos para número.")
    add_check(audit, "ano_mes_invalid_month (mês fora 01-12)", invalid_month.sum(), total_rows,
              details="Ex.: 202513 tem mês=13 (inválido).")
    add_check(audit, f"ano_mes_out_of_expected_range ({min_expected}-{max_expected})", out_of_range.sum(), total_rows,
              details="Fora do intervalo esperado do case.")

    print("\n✅ ano_mes inválidos por mês (amostra):")
    display(df_contratos.loc[invalid_month, ["contract_id","ano_mes"]].head(10))

    print("\n✅ ano_mes fora do range (amostra):")
    display(df_contratos.loc[out_of_range, ["contract_id","ano_mes"]].head(10))
else:
    add_check(audit, "ano_mes_column_missing", bad_count=total_rows, total_count=total_rows, details="Coluna ano_mes ausente")

# ---------------------------
# 3) Integridade referencial: product_id e location_id
# ---------------------------
if "product_id" in df_contratos.columns and "product_id" in df_produto.columns:
    fact_prod = df_contratos["product_id"]
    dim_prod_ids = set(df_produto["product_id"].dropna().unique().tolist())
    invalid_prod_fk = fact_prod.notna() & (~fact_prod.isin(dim_prod_ids))
    add_check(audit, "invalid_product_id_fk", invalid_prod_fk.sum(), total_rows,
              details="product_id presente na fato mas inexistente na dim_produto.")
    print("\n✅ product_id FK inválida (amostra):")
    display(df_contratos.loc[invalid_prod_fk, ["contract_id","product_id"]].head(10))
else:
    add_check(audit, "product_id_fk_check_skipped", bad_count=0, total_count=total_rows,
              details="Colunas ausentes para validar product_id.")

if "location_id" in df_contratos.columns and "location_id" in df_localidade.columns:
    fact_loc = df_contratos["location_id"]
    dim_loc_ids = set(df_localidade["location_id"].dropna().unique().tolist())
    invalid_loc_fk = fact_loc.notna() & (~fact_loc.isin(dim_loc_ids))
    add_check(audit, "invalid_location_id_fk", invalid_loc_fk.sum(), total_rows,
              details="location_id presente na fato mas inexistente na dim_localidade.")
    print("\n✅ location_id FK inválida (amostra):")
    display(df_contratos.loc[invalid_loc_fk, ["contract_id","location_id"]].head(10))
else:
    add_check(audit, "location_id_fk_check_skipped", bad_count=0, total_count=total_rows,
              details="Colunas ausentes para validar location_id.")

# ---------------------------
# 4) Unicidade de contract_id
# ---------------------------
if "contract_id" in df_contratos.columns:
    dup_contract = df_contratos["contract_id"].duplicated(keep=False)
    add_check(audit, "duplicate_contract_id", dup_contract.sum(), total_rows,
              details="Contagem de linhas que fazem parte de duplicidades (não apenas 'duplicados').")
    print("\n✅ contract_id duplicados (amostra):")
    display(df_contratos.loc[dup_contract, ["contract_id","bank","ano_mes","product_id","location_id"]].head(15))
else:
    add_check(audit, "contract_id_column_missing", bad_count=total_rows, total_count=total_rows, details="Coluna contract_id ausente")

# ---------------------------
# 5) Regras numéricas e lógicas (se as colunas existirem)
# ---------------------------
# risk_score 0-1
if "risk_score" in df_contratos.columns:
    rs = pd.to_numeric(df_contratos["risk_score"], errors="coerce")
    invalid_rs = rs.isna() | (rs < 0) | (rs > 1)
    add_check(audit, "risk_score_outside_0_1_or_nan", invalid_rs.sum(), total_rows,
              details="risk_score deve estar entre 0 e 1 após normalização.")
    print("\n✅ risk_score inválido (amostra):")
    display(df_contratos.loc[invalid_rs, ["contract_id","risk_score"]].head(10))
else:
    add_check(audit, "risk_score_check_skipped", 0, total_rows, details="Coluna risk_score ausente")

# delinquent rule: dpd < 30 => delinquent_amount_30p deve ser 0
if "dpd" in df_contratos.columns and "delinquent_amount_30p" in df_contratos.columns:
    dpd = pd.to_numeric(df_contratos["dpd"], errors="coerce")
    da30 = pd.to_numeric(df_contratos["delinquent_amount_30p"], errors="coerce")

    invalid_dpd = dpd.isna() | (dpd < 0)
    add_check(audit, "dpd_negative_or_nan", invalid_dpd.sum(), total_rows,
              details="dpd não pode ser negativo e deve ser numérico.")

    invalid_delinquent = da30.isna() | (da30 < 0)
    add_check(audit, "delinquent_amount_30p_negative_or_nan", invalid_delinquent.sum(), total_rows,
              details="delinquent_amount_30p não pode ser negativo e deve ser numérico.")

    rule_violation = (~dpd.isna()) & (~da30.isna()) & (dpd < 30) & (da30 > 0)
    add_check(audit, "rule_violation_dpd_lt_30_with_delinquent_gt_0", rule_violation.sum(), total_rows,
              details="Regra: dpd < 30 não pode ter delinquent_amount_30p > 0.")

    print("\n✅ Violações dpd<30 e delinquent_amount_30p>0 (amostra):")
    display(df_contratos.loc[rule_violation, ["contract_id","dpd","delinquent_amount_30p"]].head(10))
else:
    add_check(audit, "delinquency_rule_check_skipped", 0, total_rows, details="Colunas dpd/delinquent_amount_30p ausentes")

# ---------------------------
# 6) Salvar auditoria pré-limpeza (evidência)
# ---------------------------
audit_df = pd.DataFrame(audit).sort_values(["bad_count","check"], ascending=[False, True])
display(audit_df)

pre_path = LOG_DIR / "quality_profile_pre_v1.csv"
audit_df.to_csv(pre_path, index=False)
print("\n✅ Auditoria pré-limpeza salva em:", pre_path)


✅ Valores únicos de bank: ['Banco S.H.I.E.L.D', 'Banco Shield', 'HIDRA', 'Hidra', 'Hydra']

✅ ano_mes inválidos por mês (amostra):


,contract_id,ano_mes
19,C202501-BA-000020,202513
613,C202502-BA-000614,202513
796,C202502-HI-000797,202513
1026,C202503-BA-001027,202513
1112,C202503-BA-001113,202513
1311,C202503-HI-001312,202513
1728,C202504-BA-001729,202513
1774,C202504-HI-001775,202513
1922,C202504-HI-001923,202513
2444,C202505-HI-002445,202513



✅ ano_mes fora do range (amostra):


,contract_id,ano_mes
19,C202501-BA-000020,202513
613,C202502-BA-000614,202513
796,C202502-HI-000797,202513
1026,C202503-BA-001027,202513
1112,C202503-BA-001113,202513
1311,C202503-HI-001312,202513
1728,C202504-BA-001729,202513
1774,C202504-HI-001775,202513
1922,C202504-HI-001923,202513
2444,C202505-HI-002445,202513



✅ product_id FK inválida (amostra):


...

,contract_id,risk_score



✅ Violações dpd<30 e delinquent_amount_30p>0 (amostra):


,contract_id,dpd,delinquent_amount_30p


,check,bad_count,total_count,bad_pct,details
6,duplicate_contract_id,32,6000,0.005333,Contagem de linhas que fazem parte de duplicid...
2,ano_mes_invalid_month (mês fora 01-12),21,6000,0.003500,Ex.: 202513 tem mês=13 (inválido).
3,ano_mes_out_of_expected_range (202501-202512),21,6000,0.003500,Fora do intervalo esperado do case.
5,invalid_location_id_fk,10,6000,0.001667,location_id presente na fato mas inexistente n...
0,bank_domain_variants (apenas informativo),3,6000,0.000500,Valores únicos encontrados: ['Banco S.H.I.E.L....
1,ano_mes_invalid_format (NaN/non-numeric),0,6000,0.000000,Valores que não puderam ser convertidos para n...
9,delinquent_amount_30p_negative_or_nan,0,6000,0.000000,delinquent_amount_30p não pode ser negativo e ...
8,dpd_negative_or_nan,0,6000,0.000000,dpd não pode ser negativo e deve ser numérico.
4,invalid_product_id_fk,0,6000,0.000000,product_id presente na fato mas inexistente na...
7,risk_score_outside_0_1_or_nan,0,6000,0.000000,risk_score deve estar entre 0 e 1 após normali...



✅ Auditoria pré-limpeza salva em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/quality_profile_pre_v1.csv


## PASSO 3 — Normalização do campo `bank` e correção de `ano_mes` inválido (com trilha de auditoria)

**Objetivo deste passo**  
Corrigir dois problemas de qualidade identificados na auditoria pré-limpeza:

1) **Normalizar o domínio de `bank`** para reduzir variações textuais (ex.: "HIDRA", "Hydra", "Hidra") e garantir consistência no Power BI.  
2) **Corrigir valores inválidos de `ano_mes`** (ex.: `202513`, mês 13), aproveitando o padrão do `contract_id` como fonte de verdade do período.

**Por que fazer isso agora (antes de remover duplicidades e FKs inválidas)**  
- A normalização de `bank` evita métricas duplicadas no dashboard (um mesmo banco aparecendo como 3 categorias diferentes).  
- Corrigir `ano_mes` antes de deduplicar melhora a consistência temporal e evita que regras futuras se apoiem em um valor inválido.  
- Separar por etapas facilita auditoria: o avaliador consegue ver claramente **o que foi mudado em cada passo**.

**Como a correção será feita**
- `bank`: aplicaremos uma padronização robusta (remoção de espaços/pontuação + comparação por palavras-chave) e mapear para:
  - `Banco Shield`
  - `Hidra`
- `ano_mes`: quando `ano_mes` estiver inválido (mês fora de 01–12) ou fora do range esperado (202501–202512), extrairemos `YYYYMM` do início do `contract_id` (padrão `CYYYYMM-...`) e substituiremos o valor.

**Evidências geradas**
- Um log detalhado de mudanças (`cleaning_audit_step3_v1.csv`) com:
  - campo alterado (`bank` ou `ano_mes`)
  - valor anterior
  - valor novo
  - justificativa
- Um arquivo intermediário tratado (`fato_contratos_step3_v1.csv`) para rastrear evolução do pipeline.


In [5]:
# =========================================================
# PASSO 3 — bank (Banco Shield / Hidra) + ano_mes
# =========================================================

# IMPORTANTE:
# Para refazer corretamente, vamos partir do df_contratos (original carregado do raw),
# evitando reaproveitar df_step3 antigo.
df_step3 = df_contratos.copy()

change_log = []

def log_change(contract_id, field, old, new, reason):
    change_log.append({
        "contract_id": contract_id,
        "field": field,
        "old_value": old,
        "new_value": new,
        "reason": reason
    })

# ---------------------------------------------------------
# 1) NORMALIZAR bank PARA APENAS: "Banco Shield" e "Hidra"
# ---------------------------------------------------------
def normalize_bank_allowed(value: str) -> str:
    """
    Normaliza o campo bank para o domínio permitido:
    - "Banco Shield"
    - "Hidra"

    Estratégia:
    - strip
    - remove pontuação e espaços
    - usa palavras-chave (SHIELD, HIDRA/HYDRA)
    """
    if pd.isna(value):
        return value

    raw = str(value).strip()
    key = re.sub(r"[^A-Za-z0-9]", "", raw).upper()  # remove pontuação e espaços

    if "SHIELD" in key:
        return "Banco Shield"
    if "HIDRA" in key or "HYDRA" in key:
        return "Hidra"

    # Se cair aqui, é um valor fora do domínio esperado.
    # Mantemos o original para não "inventar" categoria,
    # e esse caso será detectável na auditoria pós-limpeza.
    return raw

if "bank" in df_step3.columns:
    old_bank = df_step3["bank"].copy()
    df_step3["bank"] = df_step3["bank"].apply(normalize_bank_allowed)

    changed_bank = old_bank.astype(str) != df_step3["bank"].astype(str)
    for idx in df_step3.index[changed_bank]:
        log_change(
            contract_id=df_step3.at[idx, "contract_id"] if "contract_id" in df_step3.columns else f"row_{idx}",
            field="bank",
            old=str(old_bank.at[idx]),
            new=str(df_step3.at[idx, "bank"]),
            reason="Normalização de domínio: padronização para valores permitidos (Banco Shield / Hidra)."
        )

    print("✅ bank: valores únicos ANTES:", sorted(old_bank.dropna().astype(str).str.strip().unique().tolist()))
    print("✅ bank: valores únicos DEPOIS:", sorted(df_step3["bank"].dropna().astype(str).str.strip().unique().tolist()))

    # Checagem rápida: existe algo fora do domínio permitido?
    allowed = {"Banco Shield", "Hidra"}
    outside = sorted(set(df_step3["bank"].dropna().unique().tolist()) - allowed)
    print("✅ bank fora do domínio permitido (se existir):", outside)
else:
    print("⚠️ Coluna 'bank' não encontrada. Pulando normalização.")

# ---------------------------------------------------------
# 2) CORRIGIR ano_mes INVÁLIDO / FORA DO RANGE VIA contract_id
# ---------------------------------------------------------
def extract_ym_from_contract_id(contract_id: str):
    """
    Extrai YYYYMM do contract_id no padrão CYYYYMM-...
    Retorna int (YYYYMM) ou np.nan se não conseguir extrair.
    """
    if pd.isna(contract_id):
        return np.nan
    m = re.match(r"^C(\d{6})-", str(contract_id).strip())
    if not m:
        return np.nan
    return int(m.group(1))

if "ano_mes" in df_step3.columns and "contract_id" in df_step3.columns:
    ano_mes = pd.to_numeric(df_step3["ano_mes"], errors="coerce")
    month = (ano_mes % 100)

    min_expected = 202501
    max_expected = 202512

    invalid_month = ano_mes.isna() | (month < 1) | (month > 12)
    out_of_range = (~ano_mes.isna()) & ((ano_mes < min_expected) | (ano_mes > max_expected))

    needs_fix = invalid_month | out_of_range
    print(f"\n✅ Linhas com ano_mes para corrigir: {int(needs_fix.sum())}")

    extracted_ym = df_step3["contract_id"].apply(extract_ym_from_contract_id)

    new_ano_mes = ano_mes.copy()
    new_ano_mes.loc[needs_fix] = extracted_ym.loc[needs_fix]

    changed_ano = (ano_mes.astype("float") != new_ano_mes.astype("float")) & needs_fix
    for idx in df_step3.index[changed_ano]:
        log_change(
            contract_id=df_step3.at[idx, "contract_id"],
            field="ano_mes",
            old=str(df_step3.at[idx, "ano_mes"]),
            new=str(int(new_ano_mes.at[idx])) if pd.notna(new_ano_mes.at[idx]) else "NaN",
            reason="Correção de ano_mes inválido/fora do range usando YYYYMM extraído do contract_id."
        )

    # aplica (sem criar coluna nova)
    df_step3["ano_mes"] = new_ano_mes.astype("Int64")

    # revalida
    ano_mes2 = pd.to_numeric(df_step3["ano_mes"], errors="coerce")
    month2 = (ano_mes2 % 100)
    invalid_month2 = ano_mes2.isna() | (month2 < 1) | (month2 > 12)
    out_of_range2 = (~ano_mes2.isna()) & ((ano_mes2 < min_expected) | (ano_mes2 > max_expected))

    print("✅ ano_mes inválido (mês fora 01-12) DEPOIS:", int(invalid_month2.sum()))
    print(f"✅ ano_mes fora do range {min_expected}-{max_expected} DEPOIS:", int(out_of_range2.sum()))
else:
    print("⚠️ Colunas 'ano_mes' e/ou 'contract_id' não encontradas. Pulando correção de ano_mes.")

# ---------------------------------------------------------
# 3) SALVAR SAÍDAS (intermediário + log) — versão v1
# ---------------------------------------------------------
log_df = pd.DataFrame(change_log)
log_path = LOG_DIR / "cleaning_audit_step3_v1.csv"
log_df.to_csv(log_path, index=False)

out_path = PROCESSED_DIR / "fato_contratos_step3_v1.csv"
df_step3.to_csv(out_path, index=False)

print("\n✅ Log de mudanças salvo em:", log_path)
print("✅ Saída intermediária salva em:", out_path)

print("\n✅ Total de mudanças registradas:", len(log_df))
print("✅ Shape df_step3:", df_step3.shape)

display(log_df["field"].value_counts())

display(log_df.head(10))


✅ bank: valores únicos ANTES: ['Banco S.H.I.E.L.D', 'Banco Shield', 'HIDRA', 'Hidra', 'Hydra']
✅ bank: valores únicos DEPOIS: ['Banco Shield', 'Hidra']
✅ bank fora do domínio permitido (se existir): []

✅ Linhas com ano_mes para corrigir: 21
✅ ano_mes inválido (mês fora 01-12) DEPOIS: 0
✅ ano_mes fora do range 202501-202512 DEPOIS: 0

✅ Log de mudanças salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/cleaning_audit_step3_v1.csv
✅ Saída intermediária salva em: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_step3_v1.csv

✅ Total de mudanças registradas: 41
✅ Shape df_step3: (6000, 11)


,count
field,
ano_mes,21
bank,20


,contract_id,field,old_value,new_value,reason
0,C202502-BA-000532,bank,HIDRA,Hidra,Normalização de domínio: padronização para val...
1,C202502-BA-000552,bank,HIDRA,Hidra,Normalização de domínio: padronização para val...
2,C202502-BA-000732,bank,Hydra,Hidra,Normalização de domínio: padronização para val...
3,C202502-HI-000902,bank,Hydra,Hidra,Normalização de domínio: padronização para val...
4,C202502-HI-000972,bank,Hydra,Hidra,Normalização de domínio: padronização para val...
5,C202502-HI-000997,bank,Hydra,Hidra,Normalização de domínio: padronização para val...
6,C202503-HI-001273,bank,HIDRA,Hidra,Normalização de domínio: padronização para val...
7,C202503-HI-001416,bank,HIDRA,Hidra,Normalização de domínio: padronização para val...
8,C202504-HI-001927,bank,Banco S.H.I.E.L.D,Banco Shield,Normalização de domínio: padronização para val...
9,C202505-BA-002077,bank,Banco S.H.I.E.L.D,Banco Shield,Normalização de domínio: padronização para val...


## PASSO 4 — Remoção de linhas com `location_id` inválido (FK) + trilha de auditoria

**Objetivo deste passo**  
Garantir **integridade referencial** no modelo estrela removendo contratos cujo `location_id` **não existe** na tabela `dim_localidade` (ex.: `999`).

**Por que isso é necessário**
- No Power BI, uma FK inválida cria **registros “sem correspondência”** na dimensão, gerando visuais por UF/cidade com valores incompletos, filtros inconsistentes e possíveis diferenças entre totais da fato e totais agregados por dimensão.
- Para análises confiáveis (principalmente por **geografia**), a fato deve referenciar apenas chaves existentes na dimensão.

**Estratégia adotada (com justificativa)**
Existem duas abordagens comuns:
1) **Remover linhas com FK inválida** (mantém o modelo consistente; reduz volume, mas aumenta confiança).
2) Corrigir para um valor “default” (preserva volume, mas introduz um dado “artificial”).

Para este projeto, vamos adotar a abordagem **(1)**: **remover** as linhas com `location_id` inválido, pois:
- Evita a criação de registros artificiais de localidade.
- Mantém a consistência do modelo e a confiança na análise geográfica.
- Deixa explícito no log e no README quantas linhas foram descartadas e por quê.

**Evidências geradas**
- `cleaning_audit_step4_v1.csv`: log das linhas removidas por FK inválida.
- `fato_contratos_step4_v1.csv`: dataset intermediário após remoção.
- Métricas de impacto: linhas antes/depois e percentual removido.


In [6]:
# =========================================================
# PASSO 4 — Remover linhas com location_id inválido (FK)
# (sem criar colunas) + trilha de auditoria
# =========================================================

# 0) Partimos sempre do último dataset válido
STEP3_PATH = PROCESSED_DIR / "fato_contratos_step3_v1.csv"
df_step4 = pd.read_csv(STEP3_PATH, low_memory=False)

# 1) Carrega dimensão de localidade (fonte de verdade)
dim_loc_path = RAW_DIR / "dim_localidade.csv"
df_localidade = pd.read_csv(dim_loc_path, low_memory=False)

valid_location_ids = set(df_localidade["location_id"].dropna().astype(int).unique().tolist())

# 2) Identifica linhas com FK inválida
# - inválido se: NaN / não numérico / não existe em dim_localidade
loc_series = pd.to_numeric(df_step4["location_id"], errors="coerce")
invalid_fk_mask = loc_series.isna() | (~loc_series.astype("Int64").isin(valid_location_ids))

invalid_count = int(invalid_fk_mask.sum())
total_before = int(len(df_step4))
print("✅ Linhas com location_id inválido:", invalid_count)
print("✅ Total de linhas ANTES:", total_before)

# 3) Log das remoções (linha a linha)
change_log = []

def log_removal(contract_id, old_location_id, reason):
    change_log.append({
        "contract_id": contract_id,
        "field": "location_id",
        "old_value": old_location_id,
        "action": "ROW_REMOVED",
        "reason": reason
    })

if invalid_count > 0:
    invalid_rows = df_step4.loc[invalid_fk_mask, ["contract_id", "location_id"]].copy()

    for _, row in invalid_rows.iterrows():
        log_removal(
            contract_id=row["contract_id"],
            old_location_id=str(row["location_id"]),
            reason="FK inválida: location_id inexistente em dim_localidade. Linha removida para preservar integridade referencial."
        )

# 4) Remove as linhas inválidas
df_step4_clean = df_step4.loc[~invalid_fk_mask].copy()

total_after = int(len(df_step4_clean))
removed_pct = (invalid_count / total_before) if total_before else 0.0

print("✅ Total de linhas DEPOIS:", total_after)
print(f"✅ Percentual removido: {removed_pct:.4%}")

# 5) Revalida (garantia)
loc_series2 = pd.to_numeric(df_step4_clean["location_id"], errors="coerce")
invalid_fk_mask2 = loc_series2.isna() | (~loc_series2.astype("Int64").isin(valid_location_ids))
print("✅ Linhas com location_id inválido (DEPOIS):", int(invalid_fk_mask2.sum()))

# 6) Salva outputs do passo 4
log_df = pd.DataFrame(change_log)
log_path = LOG_DIR / "cleaning_audit_step4_v1.csv"
out_path = PROCESSED_DIR / "fato_contratos_step4_v1.csv"

log_df.to_csv(log_path, index=False)
df_step4_clean.to_csv(out_path, index=False)

print("\n✅ Log salvo em:", log_path)
print("✅ Dataset salvo em:", out_path)
print("✅ Total de remoções registradas:", len(log_df))
print("✅ Shape df_step4_clean:", df_step4_clean.shape)

display(log_df.head(10))


✅ Linhas com location_id inválido: 10
✅ Total de linhas ANTES: 6000
✅ Total de linhas DEPOIS: 5990
✅ Percentual removido: 0.1667%
✅ Linhas com location_id inválido (DEPOIS): 0

✅ Log salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/cleaning_audit_step4_v1.csv
✅ Dataset salvo em: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_step4_v1.csv
✅ Total de remoções registradas: 10
✅ Shape df_step4_clean: (5990, 11)


,contract_id,field,old_value,action,reason
0,C202504-BA-001621,location_id,999,ROW_REMOVED,FK inválida: location_id inexistente em dim_lo...
1,C202505-BA-002143,location_id,999,ROW_REMOVED,FK inválida: location_id inexistente em dim_lo...
2,C202505-BA-002221,location_id,999,ROW_REMOVED,FK inválida: location_id inexistente em dim_lo...
3,C202506-BA-002613,location_id,999,ROW_REMOVED,FK inválida: location_id inexistente em dim_lo...
4,C202506-HI-002916,location_id,999,ROW_REMOVED,FK inválida: location_id inexistente em dim_lo...
5,C202507-BA-003030,location_id,999,ROW_REMOVED,FK inválida: location_id inexistente em dim_lo...
6,C202508-BA-003669,location_id,999,ROW_REMOVED,FK inválida: location_id inexistente em dim_lo...
7,C202509-HI-004453,location_id,999,ROW_REMOVED,FK inválida: location_id inexistente em dim_lo...
8,C202510-BA-004534,location_id,999,ROW_REMOVED,FK inválida: location_id inexistente em dim_lo...
9,C202510-HI-004980,location_id,999,ROW_REMOVED,FK inválida: location_id inexistente em dim_lo...


## PASSO 5 — Tratamento de `contract_id` duplicado (manter 1ª ocorrência e gerar novos IDs válidos para as duplicatas) + trilha de auditoria

**Objetivo deste passo**  
Eliminar duplicidades de `contract_id` **sem remover linhas**, mantendo a **primeira ocorrência** de cada `contract_id` e gerando um **novo `contract_id` válido** para cada linha duplicada subsequente.

**Por que isso é necessário**
- `contract_id` duplicado distorce qualquer métrica que conte contratos (ex.: quantidade de contratos, inadimplência por contrato, etc.).
- Remover linhas pode reduzir o dataset e perder informação. Aqui, a estratégia é **preservar todas as linhas**, tornando os identificadores únicos.

**Regra de geração do novo `contract_id` (conforme solicitado)**
Para cada linha duplicada (exceto a primeira ocorrência), gerar:

`C` + `ano_mes` + `-` + `<sigla_bank>` + `-` + `<sequencial_6_dígitos>`

Onde:
- `<sigla_bank>`:
  - `BA` para `Banco Shield`
  - `HI` para `Hidra`
- `<sequencial_6_dígitos>`:
  - calculado como **último contrato existente** (maior sequencial) **para o mesmo par (`ano_mes`, `<sigla_bank>`)** + 1  
  - e incrementa (+1) conforme surgem novas duplicatas nesse mesmo par.
  
**Importante (coerência com etapas anteriores)**
- O novo ID **preserva o `ano_mes` da linha**, mantendo consistência temporal e alinhamento com a regra usada antes (corrigir `ano_mes` via `contract_id`).
- Não criamos colunas novas: apenas atualizamos o campo `contract_id` onde necessário.

**Evidências geradas**
- `cleaning_audit_step5_v1.csv`: log das alterações de `contract_id` (antigo → novo) para cada duplicata
- `fato_contratos_step5_v1.csv`: dataset intermediário com `contract_id` únicos
- Métricas: quantidade de duplicatas encontradas, quantos IDs foram gerados, validação de unicidade final


In [7]:
# =========================================================
# PASSO 5 — Resolver duplicidade de contract_id gerando novos IDs
# Regra:
#   - manter a 1ª ocorrência de cada contract_id
#   - para duplicatas: C + ano_mes + "-" + (BA/HI) + "-" + sequencial_6dig (max+1 por (ano_mes, bank))
# =========================================================

# 0) Carrega dataset do passo anterior
STEP4_PATH = PROCESSED_DIR / "fato_contratos_step4_v1.csv"
df_step5 = pd.read_csv(STEP4_PATH, low_memory=False)

total_before = len(df_step5)
print("✅ Total de linhas ANTES:", total_before)

# 1) Mapeia bank -> sigla
bank_map = {
    "Banco Shield": "BA",
    "Hidra": "HI"
}

# valida domínio esperado
banks_found = sorted(df_step5["bank"].dropna().astype(str).str.strip().unique().tolist())
print("✅ Valores únicos de bank encontrados:", banks_found)

unknown_banks = [b for b in banks_found if b not in bank_map]
if unknown_banks:
    raise ValueError(f"Encontrados banks fora do domínio esperado: {unknown_banks}. Ajuste o mapeamento bank_map.")

# 2) Identifica duplicados mantendo a primeira ocorrência
# dup_rank = 0 para primeira ocorrência, >0 para duplicatas
dup_rank = df_step5.groupby("contract_id").cumcount()
dup_mask = dup_rank > 0

dup_rows = int(dup_mask.sum())
dup_contracts = int(df_step5.loc[df_step5.duplicated("contract_id", keep=False), "contract_id"].nunique())

print("✅ contract_id duplicados (qtd contratos com duplicidade):", dup_contracts)
print("✅ Total de linhas duplicadas (que precisarão de novo ID):", dup_rows)

# Se não houver duplicatas, salva e termina
change_log = []

def log_change(old_id, new_id, contract_id_original, reason):
    change_log.append({
        "contract_id_original": contract_id_original,
        "field": "contract_id",
        "old_value": old_id,
        "new_value": new_id,
        "reason": reason
    })

# 3) Função para extrair sequencial (6 dígitos) do contract_id atual
# Padrão esperado: CYYYYMM-BA-000001 (ou HI)
pat = re.compile(r"^C(?P<ym>\d{6})-(?P<code>BA|HI)-(?P<num>\d{6})$")

def parse_seq(cid: str):
    if pd.isna(cid):
        return None
    m = pat.match(str(cid).strip())
    if not m:
        return None
    return {
        "ym": int(m.group("ym")),
        "code": m.group("code"),
        "num": int(m.group("num"))
    }

# 4) Construir base de máximos por (ano_mes, sigla_bank) usando IDs existentes válidos
max_seq = {}  # (ym, code) -> max_num

for cid in df_step5["contract_id"].dropna().astype(str):
    info = parse_seq(cid)
    if info is None:
        continue
    key = (info["ym"], info["code"])
    max_seq[key] = max(max_seq.get(key, 0), info["num"])

# 5) Preparar contador "next" por (ano_mes, code) = max+1
next_seq = {k: v + 1 for k, v in max_seq.items()}

def get_next_seq(ym: int, code: str) -> int:
    key = (ym, code)
    if key not in next_seq:
        # caso não exista nenhum contrato prévio com esse par (ym, code)
        next_seq[key] = 1
    val = next_seq[key]
    next_seq[key] += 1
    return val

# 6) Gerar novos IDs para duplicatas (em ordem de aparição)
#    Regra: C + ano_mes + "-" + code + "-" + NNNNNN
if dup_rows > 0:
    # garante ano_mes numérico
    df_step5["ano_mes"] = pd.to_numeric(df_step5["ano_mes"], errors="coerce").astype("Int64")

    # índices das duplicatas na ordem do dataset (determinístico)
    dup_indices = df_step5.index[dup_mask].tolist()

    for idx in dup_indices:
        old_id = str(df_step5.at[idx, "contract_id"]).strip()
        ym = int(df_step5.at[idx, "ano_mes"])
        code = bank_map[str(df_step5.at[idx, "bank"]).strip()]

        seq = get_next_seq(ym, code)
        new_id = f"C{ym}-{code}-{seq:06d}"

        # proteção extra: evita colisão com algum ID já existente no dataset
        # (muito improvável pois estamos sempre acima do max, mas garante robustez)
        while (df_step5["contract_id"] == new_id).any():
            seq = get_next_seq(ym, code)
            new_id = f"C{ym}-{code}-{seq:06d}"

        df_step5.at[idx, "contract_id"] = new_id
        log_change(
            old_id=old_id,
            new_id=new_id,
            contract_id_original=old_id,
            reason="Deduplicação: mantida 1ª ocorrência; duplicata recebeu novo contract_id seguindo regra C+ano_mes+-(BA/HI)+-sequencial(6d) com base no último+1 por (ano_mes, bank)."
        )

# 7) Valida unicidade final
dup_after = int(df_step5.duplicated(subset=["contract_id"]).sum())
print("\n✅ Duplicados restantes (DEPOIS):", dup_after)

# sanity check: quantidade de mudanças deve bater com número de duplicatas tratadas
print("✅ Total de contract_id regenerados:", len(change_log))

# 8) Salvar outputs do passo 5
log_df = pd.DataFrame(change_log)
log_path = LOG_DIR / "cleaning_audit_step5_v1.csv"
out_path = PROCESSED_DIR / "fato_contratos_step5_v1.csv"

log_df.to_csv(log_path, index=False)
df_step5.to_csv(out_path, index=False)

print("\n✅ Log salvo em:", log_path)
print("✅ Dataset salvo em:", out_path)
print("✅ Shape df_step5:", df_step5.shape)

display(log_df.head(15))


✅ Total de linhas ANTES: 5990
✅ Valores únicos de bank encontrados: ['Banco Shield', 'Hidra']
✅ contract_id duplicados (qtd contratos com duplicidade): 16
✅ Total de linhas duplicadas (que precisarão de novo ID): 16

✅ Duplicados restantes (DEPOIS): 0
✅ Total de contract_id regenerados: 16

✅ Log salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/cleaning_audit_step5_v1.csv
✅ Dataset salvo em: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_step5_v1.csv
✅ Shape df_step5: (5990, 11)


,contract_id_original,field,old_value,new_value,reason
0,C202501-BA-000243,contract_id,C202501-BA-000243,C202501-BA-000261,Deduplicação: mantida 1ª ocorrência; duplicata...
1,C202502-BA-000652,contract_id,C202502-BA-000652,C202502-BA-000761,Deduplicação: mantida 1ª ocorrência; duplicata...
2,C202503-BA-001249,contract_id,C202503-BA-001249,C202503-BA-001261,Deduplicação: mantida 1ª ocorrência; duplicata...
3,C202504-BA-001682,contract_id,C202504-BA-001682,C202504-BA-001761,Deduplicação: mantida 1ª ocorrência; duplicata...
4,C202504-BA-001709,contract_id,C202504-BA-001709,C202504-BA-001762,Deduplicação: mantida 1ª ocorrência; duplicata...
5,C202505-HI-002319,contract_id,C202505-HI-002319,C202505-HI-002501,Deduplicação: mantida 1ª ocorrência; duplicata...
6,C202505-HI-002495,contract_id,C202505-HI-002495,C202505-HI-002502,Deduplicação: mantida 1ª ocorrência; duplicata...
7,C202506-BA-002586,contract_id,C202506-BA-002586,C202506-BA-002761,Deduplicação: mantida 1ª ocorrência; duplicata...
8,C202506-BA-002708,contract_id,C202506-BA-002708,C202506-BA-002762,Deduplicação: mantida 1ª ocorrência; duplicata...
9,C202506-HI-002977,contract_id,C202506-HI-002977,C202506-HI-003001,Deduplicação: mantida 1ª ocorrência; duplicata...


## PASSO 6 — Validação completa (100% alinhada ao `metadados.txt`) + geração do relatório pós-limpeza

**Objetivo deste passo**  
Realizar uma auditoria final para confirmar que o dataset tratado atende integralmente às regras e expectativas descritas no arquivo `metadados.txt`, deixando evidências claras para avaliação (logs de qualidade).

> Importante: **este passo não corrige nada**. Ele apenas mede e registra violações.  
> Caso existam violações, a correção será feita no **PASSO 7**, com trilha de auditoria (antes/depois).

---

### Requisitos do `metadados.txt` validados aqui

#### 1) Integridade referencial (FK)
- `product_id` deve existir em `dim_produto`
- `location_id` deve existir em `dim_localidade`

**Por que validar?**  
Evita “buracos” no Power BI (linhas que não casam com dimensões, filtros quebrando e categorias faltando).

---

#### 2) Domínios válidos
- `bank` deve ser apenas **"Banco Shield"** ou **"Hidra"**
- `ano_mes` deve estar no intervalo **202501 a 202512** e ter mês válido (01–12)

**Por que validar?**  
Garante consistência temporal e categórica, evitando duplicação de métricas e períodos inválidos.

---

#### 3) Regras de valores (qualidade numérica)
- `financed_amount` **não pode ser negativo**
- `outstanding_balance` **não pode ser negativo**
- `risk_score` deve estar entre **0 e 1**
- `dpd` não pode ser negativo
- `delinquent_amount_30p` não pode ser negativo
- Regra do case: se `dpd < 30`, então `delinquent_amount_30p` deve ser 0

**Por que validar?**  
Garante coerência operacional e evita análises incorretas (ex.: saldo negativo ou risco fora do intervalo).

---

#### 4) Unicidade
- `contract_id` deve ser **único**

**Por que validar?**  
Evita contagem duplicada de contratos e distorção de KPIs.

---

#### 5) Regra do dataset: “1 contrato por linha”
- `units` deve ser **inteiro** e **igual a 1** em todas as linhas

**Por que validar?**  
O dataset do case representa 1 contrato por linha. Se `units` variar, KPIs como “qtd contratos” podem ficar inconsistentes.

---

### Evidências geradas
- `quality_profile_post_v1.csv`: relatório com contagem de violações por checagem.
- `fato_contratos_clean_v1.csv`: cópia da base (após passo 5) para consumo no Power BI.

> Se qualquer checagem tiver violações (`bad_count > 0`), o PASSO 7 fará o tratamento com logs.


In [8]:
# =========================================================
# PASSO 6 — Validação final completa (metadados.txt)
# (sem criar novas colunas no dataset final)
# =========================================================

STEP5_PATH = PROCESSED_DIR / "fato_contratos_step5_v1.csv"
df_final = pd.read_csv(STEP5_PATH, low_memory=False)

# Dimensões (para validação de FKs)
df_produto = pd.read_csv(RAW_DIR / "dim_produto.csv", low_memory=False)
df_localidade = pd.read_csv(RAW_DIR / "dim_localidade.csv", low_memory=False)

valid_product_ids = set(pd.to_numeric(df_produto["product_id"], errors="coerce").dropna().astype(int).unique())
valid_location_ids = set(pd.to_numeric(df_localidade["location_id"], errors="coerce").dropna().astype(int).unique())

total = len(df_final)

def pct(x, total):
    return (x / total) if total else 0.0

checks = []

# ---------------------------------------------------------
# 1) Domínio: bank (metadados)
# ---------------------------------------------------------
allowed_banks = {"Banco Shield", "Hidra"}
bank_invalid = df_final.loc[~df_final["bank"].isin(allowed_banks), "bank"]
bank_invalid_count = int(bank_invalid.shape[0])
checks.append({
    "check": "bank_outside_allowed_domain",
    "bad_count": bank_invalid_count,
    "total_count": total,
    "bad_pct": pct(bank_invalid_count, total),
    "details": f"Valores inválidos: {sorted(bank_invalid.unique().tolist())[:10]}"
})

# ---------------------------------------------------------
# 2) ano_mes: formato + mês válido + range esperado (metadados)
# ---------------------------------------------------------
ano_mes_num = pd.to_numeric(df_final["ano_mes"], errors="coerce")
bad_format = int(ano_mes_num.isna().sum())
checks.append({
    "check": "ano_mes_invalid_format",
    "bad_count": bad_format,
    "total_count": total,
    "bad_pct": pct(bad_format, total),
    "details": "ano_mes não numérico/NaN."
})

ano_mes_int = ano_mes_num.dropna().astype(int)
months = ano_mes_int % 100
bad_month = int(((months < 1) | (months > 12)).sum())
checks.append({
    "check": "ano_mes_invalid_month",
    "bad_count": bad_month,
    "total_count": total,
    "bad_pct": pct(bad_month, total),
    "details": "Mês fora de 01-12."
})

bad_range = int(((ano_mes_int < 202501) | (ano_mes_int > 202512)).sum())
checks.append({
    "check": "ano_mes_out_of_expected_range_202501_202512",
    "bad_count": bad_range,
    "total_count": total,
    "bad_pct": pct(bad_range, total),
    "details": "Fora do intervalo 202501–202512 descrito no metadados."
})

# ---------------------------------------------------------
# 3) Integridade referencial (metadados)
# ---------------------------------------------------------
prod_series = pd.to_numeric(df_final["product_id"], errors="coerce")
bad_prod_fk = int((prod_series.isna() | (~prod_series.astype("Int64").isin(valid_product_ids))).sum())
checks.append({
    "check": "invalid_product_id_fk",
    "bad_count": bad_prod_fk,
    "total_count": total,
    "bad_pct": pct(bad_prod_fk, total),
    "details": "product_id inexistente em dim_produto ou inválido."
})

loc_series = pd.to_numeric(df_final["location_id"], errors="coerce")
bad_loc_fk = int((loc_series.isna() | (~loc_series.astype("Int64").isin(valid_location_ids))).sum())
checks.append({
    "check": "invalid_location_id_fk",
    "bad_count": bad_loc_fk,
    "total_count": total,
    "bad_pct": pct(bad_loc_fk, total),
    "details": "location_id inexistente em dim_localidade ou inválido."
})

# ---------------------------------------------------------
# 4) Unicidade: contract_id (metadados)
# ---------------------------------------------------------
dup_contracts = int(df_final.duplicated(subset=["contract_id"]).sum())
checks.append({
    "check": "duplicate_contract_id",
    "bad_count": dup_contracts,
    "total_count": total,
    "bad_pct": pct(dup_contracts, total),
    "details": "contract_id deve ser único."
})

# ---------------------------------------------------------
# 5) Regras de valores (metadados)
# ---------------------------------------------------------
# dpd >= 0
dpd = pd.to_numeric(df_final["dpd"], errors="coerce")
bad_dpd = int((dpd.isna() | (dpd < 0)).sum())
checks.append({
    "check": "dpd_negative_or_nan",
    "bad_count": bad_dpd,
    "total_count": total,
    "bad_pct": pct(bad_dpd, total),
    "details": "dpd deve ser numérico e >= 0."
})

# delinquent_amount_30p >= 0
delinq = pd.to_numeric(df_final["delinquent_amount_30p"], errors="coerce")
bad_delinq = int((delinq.isna() | (delinq < 0)).sum())
checks.append({
    "check": "delinquent_amount_30p_negative_or_nan",
    "bad_count": bad_delinq,
    "total_count": total,
    "bad_pct": pct(bad_delinq, total),
    "details": "delinquent_amount_30p deve ser numérico e >= 0."
})

# Regra: dpd < 30 => delinquent_amount_30p = 0
rule_violation = int(((dpd < 30) & (delinq > 0)).sum())
checks.append({
    "check": "rule_violation_dpd_lt_30_with_delinquent_gt_0",
    "bad_count": rule_violation,
    "total_count": total,
    "bad_pct": pct(rule_violation, total),
    "details": "Regra: dpd < 30 => delinquent_amount_30p deve ser 0."
})

# financed_amount >= 0 (metadados)
fin = pd.to_numeric(df_final["financed_amount"], errors="coerce")
bad_fin = int((fin.isna() | (fin < 0)).sum())
checks.append({
    "check": "financed_amount_negative_or_nan",
    "bad_count": bad_fin,
    "total_count": total,
    "bad_pct": pct(bad_fin, total),
    "details": "financed_amount deve ser numérico e >= 0."
})

# outstanding_balance >= 0 (metadados)
out = pd.to_numeric(df_final["outstanding_balance"], errors="coerce")
bad_out = int((out.isna() | (out < 0)).sum())
checks.append({
    "check": "outstanding_balance_negative_or_nan",
    "bad_count": bad_out,
    "total_count": total,
    "bad_pct": pct(bad_out, total),
    "details": "outstanding_balance deve ser numérico e >= 0."
})

# risk_score entre 0 e 1 (metadados)
risk = pd.to_numeric(df_final["risk_score"], errors="coerce")
bad_risk = int((risk.isna() | (risk < 0) | (risk > 1)).sum())
checks.append({
    "check": "risk_score_outside_0_1_or_nan",
    "bad_count": bad_risk,
    "total_count": total,
    "bad_pct": pct(bad_risk, total),
    "details": "risk_score deve ser numérico e estar entre 0 e 1."
})

# ---------------------------------------------------------
# 6) Regra do dataset: units = 1 por linha (metadados)
# ---------------------------------------------------------
units = pd.to_numeric(df_final["units"], errors="coerce")
# inválido se NaN, não-inteiro (ex.: 1.5) ou diferente de 1
bad_units = int((units.isna() | (units % 1 != 0) | (units.astype("Int64") != 1)).sum())
checks.append({
    "check": "units_not_integer_or_not_equal_1",
    "bad_count": bad_units,
    "total_count": total,
    "bad_pct": pct(bad_units, total),
    "details": "units deve ser inteiro e igual a 1 (1 contrato por linha)."
})

# ---------------------------------------------------------
# Relatório final + salvamentos
# ---------------------------------------------------------
quality_post = pd.DataFrame(checks).sort_values(["bad_count", "check"], ascending=[False, True])

quality_path = LOG_DIR / "quality_profile_post_v1.csv"
final_path = PROCESSED_DIR / "fato_contratos_clean_v1.csv"

quality_post.to_csv(quality_path, index=False)
df_final.to_csv(final_path, index=False)

print("✅ Total de linhas (final):", total)
print("✅ Relatório pós-limpeza salvo em:", quality_path)
print("✅ Dataset final salvo em:", final_path)

display(quality_post)


✅ Total de linhas (final): 5990
✅ Relatório pós-limpeza salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/quality_profile_post_v1.csv
✅ Dataset final salvo em: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_clean_v1.csv


,check,bad_count,total_count,bad_pct,details
10,financed_amount_negative_or_nan,30,5990,0.005008,financed_amount deve ser numérico e >= 0.
4,invalid_product_id_fk,15,5990,0.002504,product_id inexistente em dim_produto ou invál...
1,ano_mes_invalid_format,0,5990,0.000000,ano_mes não numérico/NaN.
2,ano_mes_invalid_month,0,5990,0.000000,Mês fora de 01-12.
3,ano_mes_out_of_expected_range_202501_202512,0,5990,0.000000,Fora do intervalo 202501–202512 descrito no me...
0,bank_outside_allowed_domain,0,5990,0.000000,Valores inválidos: []
8,delinquent_amount_30p_negative_or_nan,0,5990,0.000000,delinquent_amount_30p deve ser numérico e >= 0.
7,dpd_negative_or_nan,0,5990,0.000000,dpd deve ser numérico e >= 0.
6,duplicate_contract_id,0,5990,0.000000,contract_id deve ser único.
5,invalid_location_id_fk,0,5990,0.000000,location_id inexistente em dim_localidade ou i...


## PASSO 7 — Teste de consistência para `financed_amount` negativo (diagnóstico antes de corrigir)

**Objetivo deste passo**  
Antes de “corrigir” valores negativos de `financed_amount` (por exemplo, multiplicando por `-1`) ou remover linhas, precisamos **evidência** de que o problema é **apenas sinal** (erro de exportação) e não um valor que representa outro fenômeno (ex.: estorno/ajuste).

Este passo executa um **diagnóstico completo**, gerando métricas e tabelas para embasar a decisão do **PASSO 8**.

---

### O que será analisado (e por quê)

#### 1) Quantidade e impacto financeiro dos negativos
- **Por quê:** mesmo poucos registros podem distorcer KPIs (total financiado, ticket médio, evolução mensal).
- Medidas:
  - contagem de linhas com `financed_amount < 0`
  - percentual sobre o total
  - soma dos negativos, soma dos positivos e impacto se aplicarmos módulo (`abs`)

#### 2) Concentração por `bank` e por `ano_mes`
- **Por quê:** se a origem do erro for sistêmica, tende a aparecer concentrada (um banco específico, meses específicos).

#### 3) Concentração por `product_id` (e enriquecimento com `product_name`)
- **Por quê:** se o erro vier de um tipo de produto (ex.: produto com estorno), isso aparece no recorte por produto.

#### 4) Análise “produtos iguais / comportamento repetido”
- **Por quê:** se um produto específico sempre aparece com negativo em certas condições, isso sugere que o negativo pode ser **significado real**, não erro.
- Checagens:
  - produtos com maior volume de negativos
  - comparação do comportamento do mesmo produto quando positivo vs quando negativo (média, mediana)

#### 5) Testes de coerência com campos relacionados
- **Por quê:** para avaliar se negativo é “só sinal”, verificamos se o registro faz sentido frente a:
  - `outstanding_balance` (saldo em aberto)
  - `dpd` e `delinquent_amount_30p` (inadimplência)
- Checagens:
  - negativos com `outstanding_balance` muito alto (pode indicar que o valor financiado deveria ser positivo)
  - negativos com `dpd`/`delinquent_amount_30p` incoerentes
  - correlação simples e comparação de estatísticas

---

### Evidências geradas
- Um arquivo de **relatório diagnóstico** (`consistency_report_step7_v1.csv`) com indicadores consolidados.
- Tabelas exibidas no notebook (por banco, mês, produto e comportamento).

> **Importante:** este passo **não altera o dataset**. Ele apenas mede e evidencia.
> O PASSO 8 (decisão) será escrito com base nesses resultados.


In [9]:
# =========================================================
# PASSO 7 — Teste de consistência para financed_amount negativo
# (Diagnóstico sem alterar o dataset)
# =========================================================

# Entrada: usar o dataset mais atual tratado (após PASSO 6)
INPUT_PATH = PROCESSED_DIR / "fato_contratos_clean_v1.csv"

# Dimensão produto para enriquecer análises
DIM_PROD_PATH = RAW_DIR / "dim_produto.csv"

# Saída: relatório consolidado do diagnóstico
REPORT_PATH = LOG_DIR / "consistency_report_step7_v1.csv"

print("📌 Lendo:", INPUT_PATH)
df = pd.read_csv(INPUT_PATH, low_memory=False)

print("📌 Lendo dimensão de produto:", DIM_PROD_PATH)
dim_prod = pd.read_csv(DIM_PROD_PATH, low_memory=False)

# -------------------------------------------
# Preparação de tipos (sem criar colunas novas no dataset final; aqui é só para análise)
# -------------------------------------------
fin = pd.to_numeric(df["financed_amount"], errors="coerce")
outb = pd.to_numeric(df.get("outstanding_balance"), errors="coerce")
dpd = pd.to_numeric(df.get("dpd"), errors="coerce")
delinq30 = pd.to_numeric(df.get("delinquent_amount_30p"), errors="coerce")

prod = pd.to_numeric(df["product_id"], errors="coerce")
ano_mes = pd.to_numeric(df["ano_mes"], errors="coerce")

# -------------------------------------------
# 1) Quantidade e impacto financeiro
# -------------------------------------------
total_rows = len(df)
neg_mask = fin < 0
nan_mask = fin.isna()
neg_count = int(neg_mask.sum())
nan_count = int(nan_mask.sum())
neg_pct = (neg_count / total_rows) * 100 if total_rows else 0

sum_pos = float(fin[fin >= 0].sum(skipna=True))
sum_neg = float(fin[fin < 0].sum(skipna=True))  # negativo (valor < 0)
sum_abs = float(fin.abs().sum(skipna=True))
impact_if_abs = float(sum_abs - (sum_pos + sum_neg))  # quanto mudaria se tomasse módulo

summary_1 = pd.DataFrame([{
    "metric": "rows_total",
    "value": total_rows
},{
    "metric": "financed_amount_negative_count",
    "value": neg_count
},{
    "metric": "financed_amount_negative_pct",
    "value": round(neg_pct, 6)
},{
    "metric": "financed_amount_nan_count",
    "value": nan_count
},{
    "metric": "sum_financed_pos",
    "value": sum_pos
},{
    "metric": "sum_financed_neg",
    "value": sum_neg
},{
    "metric": "sum_financed_abs",
    "value": sum_abs
},{
    "metric": "impact_if_apply_abs_on_all",
    "value": impact_if_abs
}])

print("\n✅ (1) Resumo geral de financed_amount:")
display(summary_1)

# -------------------------------------------
# 2) Concentração por bank e por ano_mes
# -------------------------------------------
df_neg = df.loc[neg_mask].copy()
df_neg["financed_amount_num"] = fin[neg_mask].values
df_neg["ano_mes_num"] = ano_mes[neg_mask].values

by_bank = (df_neg
           .groupby("bank", dropna=False)["financed_amount_num"]
           .agg(neg_count="count", neg_sum="sum", neg_mean="mean", neg_min="min", neg_max="max")
           .sort_values(["neg_count", "neg_sum"], ascending=[False, True]))

by_month = (df_neg
            .groupby("ano_mes_num", dropna=False)["financed_amount_num"]
            .agg(neg_count="count", neg_sum="sum", neg_mean="mean", neg_min="min", neg_max="max")
            .sort_values(["neg_count", "neg_sum"], ascending=[False, True]))

print("\n✅ (2) Concentração de negativos por bank:")
display(by_bank.head(20))

print("\n✅ (2) Concentração de negativos por ano_mes:")
display(by_month.head(20))

# -------------------------------------------
# 3) Concentração por produto (product_id) + enriquecimento com nome/descrição
# -------------------------------------------
df_neg["product_id_num"] = pd.to_numeric(df_neg["product_id"], errors="coerce").astype("Int64")

# tenta achar um nome descritivo do produto (sem assumir o nome da coluna)
# iremos pegar a primeira coluna textual que pareça descrição/nome
text_cols = [c for c in dim_prod.columns if dim_prod[c].dtype == "object"]
name_col = None
for c in text_cols:
    if any(k in c.lower() for k in ["name", "nome", "desc", "descr", "produto"]):
        name_col = c
        break

dim_prod_merge = dim_prod.copy()
dim_prod_merge["product_id"] = pd.to_numeric(dim_prod_merge["product_id"], errors="coerce").astype("Int64")

df_neg_prod = df_neg.merge(dim_prod_merge[["product_id"] + ([name_col] if name_col else [])],
                           on="product_id", how="left")

by_product = (df_neg_prod
              .groupby(["product_id"] + ([name_col] if name_col else []), dropna=False)["financed_amount_num"]
              .agg(neg_count="count", neg_sum="sum", neg_mean="mean", neg_min="min", neg_max="max")
              .sort_values(["neg_count", "neg_sum"], ascending=[False, True]))

print("\n✅ (3) Produtos com maior incidência de financed_amount negativo:")
display(by_product.head(25))

# -------------------------------------------
# 4) Análise “produtos iguais”: comparar comportamento do mesmo produto (positivo vs negativo)
# -------------------------------------------
df_aux = df.copy()
df_aux["financed_amount_num"] = fin
df_aux["product_id_num"] = pd.to_numeric(df_aux["product_id"], errors="coerce").astype("Int64")

# Seleciona produtos que aparecem no conjunto negativo
products_with_neg = df_neg["product_id_num"].dropna().unique().tolist()

df_prod_compare = df_aux[df_aux["product_id_num"].isin(products_with_neg)].copy()

comp = (df_prod_compare
        .assign(is_negative=lambda x: x["financed_amount_num"] < 0)
        .groupby(["product_id_num", "is_negative"])["financed_amount_num"]
        .agg(count="count", sum="sum", mean="mean", median="median", min="min", max="max")
        .reset_index())

# Pivot para comparar lado a lado
comp_pivot = comp.pivot(index="product_id_num", columns="is_negative")
comp_pivot.columns = [f"{m}_{'NEG' if flag else 'POS'}" for m, flag in comp_pivot.columns]
comp_pivot = comp_pivot.reset_index()

# Enriquecer com nome do produto
if name_col:
    prod_names = dim_prod_merge[["product_id", name_col]].rename(columns={"product_id":"product_id_num"})
    comp_pivot = comp_pivot.merge(prod_names, on="product_id_num", how="left")

# Criar um “indicador” de suspeita: produto que tem NEG e POS com magnitudes muito semelhantes
# (não é prova, mas ajuda a apontar erro de sinal)
if "mean_NEG" in comp_pivot.columns and "mean_POS" in comp_pivot.columns:
    comp_pivot["abs_mean_ratio_NEG_to_POS"] = (comp_pivot["mean_NEG"].abs() / comp_pivot["mean_POS"].replace(0, np.nan))

print("\n✅ (4) Comparação por produto (POS vs NEG):")
display(comp_pivot.sort_values(by=[c for c in comp_pivot.columns if "count_NEG" in c][0], ascending=False).head(30))

# -------------------------------------------
# 5) Testes de coerência com outstanding_balance, dpd e delinquent_amount_30p
# -------------------------------------------
df_neg_checks = df.loc[neg_mask, ["contract_id", "bank", "ano_mes", "product_id", "financed_amount", "outstanding_balance", "dpd", "delinquent_amount_30p"]].copy()
df_neg_checks["financed_amount_num"] = fin[neg_mask].values
df_neg_checks["outstanding_balance_num"] = outb[neg_mask].values
df_neg_checks["dpd_num"] = dpd[neg_mask].values
df_neg_checks["delinq30_num"] = delinq30[neg_mask].values

# Indicadores simples:
# - % negativos com outstanding_balance > 0 (se financed fosse estorno, saldo poderia ser 0, mas não é regra)
neg_with_positive_balance = int((df_neg_checks["outstanding_balance_num"] > 0).sum())
neg_with_positive_balance_pct = (neg_with_positive_balance / max(1, neg_count)) * 100

# - negativos com dpd alto e delinq30 alto (pode indicar contrato ativo, sugerindo que financed deveria ser positivo)
neg_with_dpd_ge_30 = int((df_neg_checks["dpd_num"] >= 30).sum())
neg_with_delinq_gt_0 = int((df_neg_checks["delinq30_num"] > 0).sum())

coherence_summary = pd.DataFrame([{
    "metric": "neg_with_outstanding_balance_gt_0_count",
    "value": neg_with_positive_balance
},{
    "metric": "neg_with_outstanding_balance_gt_0_pct",
    "value": round(neg_with_positive_balance_pct, 6)
},{
    "metric": "neg_with_dpd_ge_30_count",
    "value": neg_with_dpd_ge_30
},{
    "metric": "neg_with_delinquent_amount_30p_gt_0_count",
    "value": neg_with_delinq_gt_0
}])

print("\n✅ (5) Indicadores de coerência (negativos x campos relacionados):")
display(coherence_summary)

print("\n✅ (5) Amostra de linhas com financed_amount negativo para inspeção manual:")
display(df_neg_checks.head(20))

# -------------------------------------------
# Consolidar relatório para salvar em CSV (resumo)
# -------------------------------------------
report_frames = []
report_frames.append(summary_1.assign(section="summary"))

report_frames.append(by_bank.reset_index().assign(section="by_bank"))
report_frames.append(by_month.reset_index().rename(columns={"ano_mes_num":"ano_mes"}).assign(section="by_month"))

# by_product pode ter índice com múltiplas colunas; reset para salvar
by_product_reset = by_product.reset_index()
by_product_reset["section"] = "by_product"
report_frames.append(by_product_reset)

report_frames.append(coherence_summary.assign(section="coherence"))

report = pd.concat(report_frames, ignore_index=True, sort=False)
report.to_csv(REPORT_PATH, index=False)

print("\n✅ Relatório consolidado salvo em:", REPORT_PATH)


📌 Lendo: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_clean_v1.csv
📌 Lendo dimensão de produto: /content/drive/MyDrive/Case/case-shield-hidra/data/raw/dim_produto.csv

✅ (1) Resumo geral de financed_amount:


,metric,value
0,rows_total,5.990000e+03
1,financed_amount_negative_count,1.400000e+01
2,financed_amount_negative_pct,2.337230e-01
3,financed_amount_nan_count,1.600000e+01
4,sum_financed_pos,8.557159e+07
5,sum_financed_neg,-3.970828e+05
6,sum_financed_abs,8.596867e+07
7,impact_if_apply_abs_on_all,7.941656e+05



✅ (2) Concentração de negativos por bank:


,neg_count,neg_sum,neg_mean,neg_min,neg_max
bank,,,,,
Hidra,8,-198297.10,-24787.137500,-74180.38,-194.90
Banco Shield,6,-198785.72,-33130.953333,-65664.66,-704.97



✅ (2) Concentração de negativos por ano_mes:


...


✅ (5) Indicadores de coerência (negativos x campos relacionados):


,metric,value
0,neg_with_outstanding_balance_gt_0_count,14.0
1,neg_with_outstanding_balance_gt_0_pct,100.0
2,neg_with_dpd_ge_30_count,0.0
3,neg_with_delinquent_amount_30p_gt_0_count,0.0



✅ (5) Amostra de linhas com financed_amount negativo para inspeção manual:


,contract_id,bank,ano_mes,product_id,financed_amount,outstanding_balance,dpd,delinquent_amount_30p,financed_amount_num,outstanding_balance_num,dpd_num,delinq30_num
267,C202501-HI-000268,Hidra,202501,1014.0,-28442.23,28051.26,0,0.0,-28442.23,28051.26,0,0.0
1220,C202503-BA-001221,Banco Shield,202503,1011.0,-56840.59,49717.17,0,0.0,-56840.59,49717.17,0,0.0
2079,C202505-BA-002081,Banco Shield,202505,1005.0,-65664.66,52406.87,0,0.0,-65664.66,52406.87,0,0.0
2244,C202505-BA-002248,Banco Shield,202505,1011.0,-54237.15,49506.71,0,0.0,-54237.15,49506.71,0,0.0
2286,C202505-HI-002290,Hidra,202505,1002.0,-194.90,203.96,0,0.0,-194.90,203.96,0,0.0
2307,C202505-HI-002311,Hidra,202505,1011.0,-74180.38,54879.65,0,0.0,-74180.38,54879.65,0,0.0
2969,C202506-HI-002975,Hidra,202506,1002.0,-891.90,556.79,0,0.0,-891.90,556.79,0,0.0
3102,C202507-BA-003109,Banco Shield,202507,1019.0,-4650.56,2944.49,0,0.0,-4650.56,2944.49,0,0.0
3284,C202507-HI-003291,Hidra,202507,1015.0,-2487.42,2378.85,0,0.0,-2487.42,2378.85,0,0.0
3322,C202507-HI-003329,Hidra,202507,1013.0,-29710.24,26397.56,0,0.0,-29710.24,26397.56,0,0.0



✅ Relatório consolidado salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/consistency_report_step7_v1.csv


## PASSO 8 — Correção de `financed_amount` negativo (tratamento por inversão de sinal) + validações finais

**Decisão adotada (baseada no PASSO 7):**  
Os 14 registros com `financed_amount < 0` apresentam indícios consistentes de **inversão de sinal** (erro de carga), pois:
- representam apenas **0,2337%** da base;
- **100%** deles possuem `outstanding_balance > 0`;
- não há evidência de inadimplência (`dpd >= 30` = 0 e `delinquent_amount_30p > 0` = 0);
- valores negativos têm magnitude plausível para financiamentos.

**Regra aplicada:**  
- Se `financed_amount` for numérico e `< 0`, então:  
  `financed_amount := abs(financed_amount)`  
- Valores `NaN` permanecem como `NaN` (não inventamos valor).
- Todas as alterações são registradas no log.

**Saídas do passo:**
- Dataset corrigido salvo em `.../processed/fato_contratos_clean_v3.csv`
- Log de correções salvo em `.../quality_logs/cleaning_audit_step8_v1.csv`
- Revalidação: `financed_amount_negative_or_nan` deve ficar apenas com **NaNs**, e negativos devem ser **0**.


In [10]:
# =========================================================
# PASSO 8 — Corrigir financed_amount negativo (inversão de sinal)
# + gerar log + revalidar check
# =========================================================

INPUT_PATH = PROCESSED_DIR / "fato_contratos_clean_v1.csv"
OUTPUT_PATH = PROCESSED_DIR / "fato_contratos_clean_v2.csv"
LOG_PATH = LOG_DIR / "cleaning_audit_step8_v1.csv"

print("📌 Lendo:", INPUT_PATH)
df = pd.read_csv(INPUT_PATH, low_memory=False)

# Garantir tipo numérico para financed_amount
df["financed_amount"] = pd.to_numeric(df["financed_amount"], errors="coerce")

# Máscara de negativos
neg_mask = df["financed_amount"] < 0
neg_count = int(neg_mask.sum())
total_rows = len(df)

print(f"✅ Negativos antes: {neg_count} ({(neg_count/total_rows)*100:.6f}%)")

# Log (somente linhas alteradas)
audit_rows = []
if neg_count > 0:
    # Guardar valores antigos
    old_vals = df.loc[neg_mask, ["contract_id", "financed_amount"]].copy()
    # Corrigir: abs()
    df.loc[neg_mask, "financed_amount"] = df.loc[neg_mask, "financed_amount"].abs()

    new_vals = df.loc[neg_mask, ["contract_id", "financed_amount"]].copy()

    # Montar auditoria
    for (_, row_old), (_, row_new) in zip(old_vals.iterrows(), new_vals.iterrows()):
        audit_rows.append({
            "contract_id": row_old["contract_id"],
            "field": "financed_amount",
            "old_value": float(row_old["financed_amount"]),
            "new_value": float(row_new["financed_amount"]),
            "action": "SIGN_FIXED_ABS",
            "reason": "Correção por evidência de inversão de sinal (Passo 7)."
        })

audit = pd.DataFrame(audit_rows)

# Revalidação pós-correção
neg_after = int((df["financed_amount"] < 0).sum())
nan_after = int(df["financed_amount"].isna().sum())

print(f"✅ Negativos depois: {neg_after}")
print(f"✅ NaN depois (mantidos): {nan_after}")

# Salvar
audit.to_csv(LOG_PATH, index=False)
df.to_csv(OUTPUT_PATH, index=False)

print("✅ Log salvo em:", LOG_PATH)
print("✅ Dataset salvo em:", OUTPUT_PATH)
print("✅ Shape final:", df.shape)

# Mini-check final
check = pd.DataFrame([{
    "check": "financed_amount_negative",
    "bad_count": neg_after,
    "total_count": total_rows,
    "bad_pct": (neg_after/total_rows)*100 if total_rows else 0,
    "details": "Após correção, financed_amount não deve ter valores < 0."
},{
    "check": "financed_amount_nan",
    "bad_count": nan_after,
    "total_count": total_rows,
    "bad_pct": (nan_after/total_rows)*100 if total_rows else 0,
    "details": "NaNs permanecem e podem ser tratados em passo separado se o metadados exigir."
}])

display(check)


📌 Lendo: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_clean_v1.csv
✅ Negativos antes: 14 (0.233723%)
✅ Negativos depois: 0
✅ NaN depois (mantidos): 16
✅ Log salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/cleaning_audit_step8_v1.csv
✅ Dataset salvo em: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_clean_v2.csv
✅ Shape final: (5990, 11)


,check,bad_count,total_count,bad_pct,details
0,financed_amount_negative,0,5990,0.000000,"Após correção, financed_amount não deve ter va..."
1,financed_amount_nan,16,5990,0.267112,NaNs permanecem e podem ser tratados em passo ...


## PASSO 9 — Tratar `financed_amount` NaN com base em `outstanding_balance` (imputação conservadora)

**Problema atual:**  
Após o Passo 8, restaram **16 valores NaN** em `financed_amount`.

**Hipótese de negócio (para imputação):**  
Em geral, o `financed_amount` (valor financiado no contrato) tende a ser **maior ou igual** ao `outstanding_balance` (saldo devedor atual).  
Logo, substituir `financed_amount` ausente por `outstanding_balance` fornece um **valor conservador** (piso), preservando coerência e evitando criar valores artificiais.

### (A) Teste de consistência (base não nula)
Antes de imputar, validaremos na base “boa”:
- % de contratos onde `outstanding_balance > financed_amount` (indicaria casos onde saldo ficou maior que o financiado, por juros/encargos/etc.)
- Estatísticas do ratio `outstanding_balance / financed_amount`

### (B) Regra de correção
Para linhas onde:
- `financed_amount` é NaN **e**
- `outstanding_balance` é numérico e `>= 0`

Aplicar:
- `financed_amount := outstanding_balance`

**Transparência:**  
- Registrar todas as alterações em um log
- Criar coluna `financed_amount_imputed` = 1 se imputado, senão 0

### Saídas
- Dataset salvo em `.../processed/fato_contratos_clean_v4.csv`
- Log salvo em `.../quality_logs/cleaning_audit_step9_v1.csv`
- Check final: `financed_amount` sem NaN (se `outstanding_balance` existir para todos os 16).


In [11]:
# =========================================================
# PASSO 9 — Imputar financed_amount (NaN) com outstanding_balance
# + teste de consistência + log
# =========================================================

INPUT_PATH = PROCESSED_DIR / "fato_contratos_clean_v2.csv"
OUTPUT_PATH = PROCESSED_DIR / "fato_contratos_clean_v3.csv"
LOG_PATH = LOG_DIR / "cleaning_audit_step9_v1.csv"

print("📌 Lendo:", INPUT_PATH)
df = pd.read_csv(INPUT_PATH, low_memory=False)

# Garantir numéricos
df["financed_amount"] = pd.to_numeric(df["financed_amount"], errors="coerce")
df["outstanding_balance"] = pd.to_numeric(df["outstanding_balance"], errors="coerce")

total_rows = len(df)

# ---------------------------
# (A) Teste de consistência na base "não nula"
# ---------------------------
base_ok = df[df["financed_amount"].notna() & df["outstanding_balance"].notna()].copy()

# Evitar divisão por zero
base_ok = base_ok[base_ok["financed_amount"] > 0]

viol = (base_ok["outstanding_balance"] > base_ok["financed_amount"]).sum()
viol_pct = (viol / len(base_ok) * 100) if len(base_ok) else 0

ratio = (base_ok["outstanding_balance"] / base_ok["financed_amount"]).replace([np.inf, -np.inf], np.nan)
ratio_desc = ratio.describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]).to_dict()

test_summary = pd.DataFrame([{
    "rows_total": total_rows,
    "rows_ok_for_test": int(len(base_ok)),
    "viol_outstanding_gt_financed_count": int(viol),
    "viol_outstanding_gt_financed_pct": float(viol_pct),
    "ratio_mean_outstanding_div_financed": float(ratio.mean(skipna=True)),
    "ratio_median_outstanding_div_financed": float(ratio.median(skipna=True)),
    "ratio_p05": float(ratio.quantile(0.05)),
    "ratio_p95": float(ratio.quantile(0.95)),
}])

print("\n✅ (A) Teste de consistência (outstanding_balance vs financed_amount):")
display(test_summary)

# ---------------------------
# (B) Imputação dos NaNs
# ---------------------------
nan_mask = df["financed_amount"].isna()
nan_count = int(nan_mask.sum())

# Só imputar quando outstanding_balance é válido e >= 0
can_impute = nan_mask & df["outstanding_balance"].notna() & (df["outstanding_balance"] >= 0)
impute_count = int(can_impute.sum())

print(f"\n✅ financed_amount NaN antes: {nan_count}")
print(f"✅ Linhas elegíveis p/ imputação via outstanding_balance: {impute_count}")

# Criar flag de imputação
df["financed_amount_imputed"] = 0

audit_rows = []
if impute_count > 0:
    old_vals = df.loc[can_impute, ["contract_id", "financed_amount", "outstanding_balance"]].copy()

    df.loc[can_impute, "financed_amount"] = df.loc[can_impute, "outstanding_balance"]
    df.loc[can_impute, "financed_amount_imputed"] = 1

    new_vals = df.loc[can_impute, ["contract_id", "financed_amount"]].copy()

    for (_, row_old), (_, row_new) in zip(old_vals.iterrows(), new_vals.iterrows()):
        audit_rows.append({
            "contract_id": row_old["contract_id"],
            "field": "financed_amount",
            "old_value": None,
            "new_value": float(row_new["financed_amount"]),
            "action": "IMPUTE_FROM_OUTSTANDING_BALANCE",
            "reason": "financed_amount ausente; imputado com outstanding_balance (valor conservador/piso).",
            "source_value_outstanding_balance": float(row_old["outstanding_balance"])
        })

audit = pd.DataFrame(audit_rows)

# Pós-imputação
nan_after = int(df["financed_amount"].isna().sum())
neg_after = int((df["financed_amount"] < 0).sum())

print(f"\n✅ financed_amount NaN depois: {nan_after}")
print(f"✅ financed_amount negativos depois: {neg_after}")

# Salvar
audit.to_csv(LOG_PATH, index=False)
df.to_csv(OUTPUT_PATH, index=False)

print("\n✅ Log salvo em:", LOG_PATH)
print("✅ Dataset salvo em:", OUTPUT_PATH)
print("✅ Shape final:", df.shape)

# Mini-check final
check = pd.DataFrame([{
    "check": "financed_amount_nan",
    "bad_count": nan_after,
    "total_count": total_rows,
    "bad_pct": (nan_after/total_rows)*100 if total_rows else 0,
    "details": "Após imputação, financed_amount não deve ficar NaN (se houver outstanding_balance disponível)."
},{
    "check": "financed_amount_negative",
    "bad_count": neg_after,
    "total_count": total_rows,
    "bad_pct": (neg_after/total_rows)*100 if total_rows else 0,
    "details": "financed_amount deve ser >= 0."
},{
    "check": "financed_amount_imputed_count",
    "bad_count": int(df["financed_amount_imputed"].sum()),
    "total_count": total_rows,
    "bad_pct": (df["financed_amount_imputed"].sum()/total_rows)*100 if total_rows else 0,
    "details": "Quantidade de linhas onde financed_amount foi imputado."
}])

display(check)


📌 Lendo: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_clean_v2.csv

✅ (A) Teste de consistência (outstanding_balance vs financed_amount):


,rows_total,rows_ok_for_test,viol_outstanding_gt_financed_count,viol_outstanding_gt_financed_pct,ratio_mean_outstanding_div_financed,ratio_median_outstanding_div_financed,ratio_p05,ratio_p95
0,5990,4734,518,10.942121,0.824777,0.825263,0.624167,1.026853



✅ financed_amount NaN antes: 16
✅ Linhas elegíveis p/ imputação via outstanding_balance: 16

✅ financed_amount NaN depois: 0
✅ financed_amount negativos depois: 0

✅ Log salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/cleaning_audit_step9_v1.csv
✅ Dataset salvo em: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_clean_v3.csv
✅ Shape final: (5990, 12)


,check,bad_count,total_count,bad_pct,details
0,financed_amount_nan,0,5990,0.000000,"Após imputação, financed_amount não deve ficar..."
1,financed_amount_negative,0,5990,0.000000,financed_amount deve ser >= 0.
2,financed_amount_imputed_count,16,5990,0.267112,Quantidade de linhas onde financed_amount foi ...


## PASSO 10 — Remover linhas com `product_id` inválido (FK quebrada)

**Problema:**  
Existem linhas em `fact_contratos` cujo `product_id` **não existe** na `dim_produto`.  
Isso quebra a integridade referencial e atrapalha qualquer modelagem estrela (Power BI / SQL).

### Objetivo
1) Identificar quais `product_id` são inválidos (não presentes em `dim_produto`)  
2) Quantificar quantas linhas serão removidas  
3) Remover essas linhas da fato  
4) Salvar:
- Dataset final pós-FK: `fato_contratos_clean_v5.csv`
- Log de remoção: `cleaning_audit_step10_v1.csv`

### Regra de remoção
Remover linhas onde:
- `product_id` é numérico **e**
- `product_id` ∉ conjunto de `product_id` existentes na `dim_produto`

> Obs.: aqui **não imputamos** produto — porque imputar mudaria o significado do contrato.  
> Para integridade e análises confiáveis, o correto é remover e registrar.


In [12]:
# =========================================================
# PASSO 10 — Remover linhas com product_id inválido (FK)
# =========================================================

BASE_DIR = Path("/content/drive/MyDrive/Case/case-shield-hidra")
PROCESSED_DIR = BASE_DIR / "data/processed"
RAW_DIR = BASE_DIR / "data/raw"
LOG_DIR = BASE_DIR / "quality_logs"

FACT_IN = PROCESSED_DIR / "fato_contratos_clean_v3.csv"
DIM_PROD = RAW_DIR / "dim_produto.csv"

FACT_OUT = PROCESSED_DIR / "fato_contratos_clean_v4.csv"  # mantém o nome do seu pipeline
LOG_OUT  = LOG_DIR / "cleaning_audit_step10_v1.csv"

print("📌 Lendo fato:", FACT_IN)
df = pd.read_csv(FACT_IN, low_memory=False)

print("📌 Lendo dimensão de produto:", DIM_PROD)
dim_prod = pd.read_csv(DIM_PROD, low_memory=False)

def to_int64(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce").replace([np.inf, -np.inf], np.nan)
    # mantém somente inteiros válidos
    s = s.where(np.isclose(s, np.floor(s), equal_nan=False))
    return s.astype("Int64")

# normaliza ids
df["product_id_int"] = to_int64(df["product_id"]) if "product_id" in df.columns else pd.Series([pd.NA]*len(df), dtype="Int64")
dim_prod["product_id_int"] = to_int64(dim_prod["product_id"]) if "product_id" in dim_prod.columns else pd.Series([], dtype="Int64")

valid_products = set(dim_prod["product_id_int"].dropna().unique().tolist())

total_before = len(df)

# (A) product_id nulo
mask_pid_nan = df["product_id_int"].isna()
nan_count = int(mask_pid_nan.sum())

# (B) product_id presente mas não existe na dimensão
mask_has_pid = df["product_id_int"].notna()
mask_invalid_fk = mask_has_pid & (~df["product_id_int"].isin(valid_products))
invalid_fk_count = int(mask_invalid_fk.sum())

# linhas a remover = NaN ou FK quebrada
mask_remove = mask_pid_nan | mask_invalid_fk
remove_count = int(mask_remove.sum())

# amostra de removidos
cols_show = [c for c in ["contract_id","bank","ano_mes","product_id","product_id_int","financed_amount","outstanding_balance"] if c in df.columns]
sample_removed = df.loc[mask_remove, cols_show].head(30)

df_out = df.loc[~mask_remove].copy()
total_after = len(df_out)

print(f"\n✅ Total antes: {total_before}")
print(f"✅ product_id NaN: {nan_count} ({nan_count/total_before*100:.6f}%)")
print(f"✅ product_id FK inválido (não existe na dim): {invalid_fk_count} ({invalid_fk_count/total_before*100:.6f}%)")
print(f"✅ Total removidas (NaN + inválidos): {remove_count}")
print(f"✅ Total depois: {total_after}")

print("\n✅ Amostra de linhas removidas:")
display(sample_removed)

# log
audit = pd.DataFrame([
    {"check":"product_id_nan", "bad_count":nan_count, "total_count":total_before, "bad_pct": nan_count/total_before*100 if total_before else 0, "details":"Linhas com product_id ausente (NA). Removidas."},
    {"check":"invalid_product_id_fk", "bad_count":invalid_fk_count, "total_count":total_before, "bad_pct": invalid_fk_count/total_before*100 if total_before else 0, "details":"Linhas com product_id fora da dim_produto. Removidas."},
    {"check":"rows_removed_step10", "bad_count":remove_count, "total_count":total_before, "bad_pct": remove_count/total_before*100 if total_before else 0, "details":"Total removido no passo 10 (NaN + FK inválidos)."},
    {"check":"rows_total_after_step10", "bad_count":total_after, "total_count":total_after, "bad_pct":0.0, "details":"Total após limpeza do passo 10."},
])

LOG_DIR.mkdir(parents=True, exist_ok=True)
audit.to_csv(LOG_OUT, index=False)
df_out.to_csv(FACT_OUT, index=False)

print("\n✅ Log salvo em:", LOG_OUT)
print("✅ Dataset salvo em:", FACT_OUT)
print("✅ Shape final:", df_out.shape)


📌 Lendo fato: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_clean_v3.csv
📌 Lendo dimensão de produto: /content/drive/MyDrive/Case/case-shield-hidra/data/raw/dim_produto.csv

✅ Total antes: 5990
✅ product_id NaN: 15 (0.250417%)
✅ product_id FK inválido (não existe na dim): 0 (0.000000%)
✅ Total removidas (NaN + inválidos): 15
✅ Total depois: 5975

✅ Amostra de linhas removidas:


,contract_id,bank,ano_mes,product_id,product_id_int,financed_amount,outstanding_balance
460,C202501-HI-000461,Hidra,202501,NaN,<NA>,58427.02,57011.90
672,C202502-BA-000673,Banco Shield,202502,NaN,<NA>,35814.85,26716.02
967,C202502-HI-000968,Hidra,202502,NaN,<NA>,2961.60,2551.00
1253,C202503-BA-001254,Banco Shield,202503,NaN,<NA>,8852.43,6766.44
1629,C202504-BA-001631,Banco Shield,202504,NaN,<NA>,737.68,690.94
1866,C202504-HI-001868,Hidra,202504,NaN,<NA>,15258.28,12066.61
1871,C202504-HI-001873,Hidra,202504,NaN,<NA>,18941.28,17825.11
1873,C202504-HI-001875,Hidra,202504,NaN,<NA>,10647.77,6499.34
2989,C202506-HI-002995,Hidra,202506,NaN,<NA>,3726.60,3057.53
3420,C202507-HI-003427,Hidra,202507,NaN,<NA>,8045.21,5633.88



✅ Log salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/cleaning_audit_step10_v1.csv
✅ Dataset salvo em: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_clean_v4.csv
✅ Shape final: (5975, 13)


## Passo 11 — Validação final de qualidade (conformidade com estrutura e integridade)

Como o arquivo `metadados.txt` não foi localizado no repositório, a validação foi executada com regras padrão de qualidade e consistência de dados, cobrindo: presença de colunas mínimas, integridade referencial (FKs), tipos e faixas de valores, completude (NaNs) e duplicidades.

### 1) Verificações executadas
- **Estrutura mínima do dataset**: confirmação de colunas essenciais (`contract_id`, `bank`, `ano_mes`, `product_id`, `financed_amount`, `outstanding_balance`).
- **Validação de data (`ano_mes`)**: conferência de formato **`YYYYMM` (6 dígitos)** e mês válido (01–12).
- **Completude (NaNs)**: checagem de valores ausentes em colunas mínimas.
- **Validação numérica**:
  - Conversão segura para numérico (`errors=coerce`).
  - Regra de não negatividade para `financed_amount` e `outstanding_balance`.
- **Integridade referencial**:
  - `product_id` da fato existe na `dim_produto`.
  - `location_id` da fato existe na `dim_localidade`.
- **Duplicidade** (regra soft): verificação de duplicação na chave candidata (`contract_id`, `ano_mes`, `bank`).
- **Faixa de score**: `risk_score` dentro de **[0,1]** (quando aplicável).

### 2) Resultados principais
- As colunas mínimas necessárias estavam presentes.
- As chaves estrangeiras de `product_id` e `location_id` apresentaram integridade (sem FKs quebradas).
- Os campos financeiros (`financed_amount` e `outstanding_balance`) estavam convertíveis para numérico e sem valores negativos.
- Foi identificada necessidade de **ajuste na validação de `ano_mes`**, pois os dados estão no padrão `YYYYMM` (e não `MM/YYYY`).
- Foi detectada a existência de valores ausentes em `product_id` em execução anterior do pipeline, o que foi endereçado no Passo 10 (remoção de chaves ausentes/ inválidas), garantindo integridade do fato para modelagem.

### 3) Evidências e rastreabilidade
- O resumo das validações (PASS/WARN/FAIL) foi exportado para:  
  `quality_logs/validation_step11_summary_v2.csv`
- Uma amostra de registros com inconsistências (quando existirem) foi exportada para:  
  `quality_logs/validation_step11_issues_sample_v2.csv`

### 4) Conclusão
Após os ajustes de formato de `ano_mes` para `YYYYMM` e a remoção de registros com `product_id` ausente ou inválido (Passo 10), o dataset final fica adequado para carga no banco e para o modelo dimensional (fato + dimensões), preservando integridade referencial e qualidade mínima para análises e dashboards.


In [13]:
# =====================================================================================
# PASSO 11 — Validação final de qualidade (conformidade com estrutura e integridade)
# =====================================================================================

BASE_DIR = Path("/content/drive/MyDrive/Case/case-shield-hidra")
PROCESSED_DIR = BASE_DIR / "data/processed"
RAW_DIR = BASE_DIR / "data/raw"
LOG_DIR = BASE_DIR / "quality_logs"

FACT_PATH = PROCESSED_DIR / "fato_contratos_clean_v4.csv"  # saída do passo 10 corrigido
DIM_PROD_PATH = RAW_DIR / "dim_produto.csv"
DIM_LOC_PATH  = RAW_DIR / "dim_localidade.csv"

OUT_SUMMARY = LOG_DIR / "validation_step11_summary_v1.csv"
OUT_ISSUES  = LOG_DIR / "validation_step11_issues_sample_v1.csv"

def to_int64(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce").replace([np.inf, -np.inf], np.nan)
    s = s.where(np.isclose(s, np.floor(s), equal_nan=False))
    return s.astype("Int64")

def add_check(checks, name, status, bad_count=None, total=None, bad_pct=None, details=""):
    checks.append({
        "check": name,
        "status": status,   # PASS / WARN / FAIL
        "bad_count": bad_count,
        "total_count": total,
        "bad_pct": bad_pct,
        "details": details
    })

print("📌 Lendo fato:", FACT_PATH)
df = pd.read_csv(FACT_PATH, low_memory=False)

print("📌 Lendo dim_produto:", DIM_PROD_PATH)
dim_prod = pd.read_csv(DIM_PROD_PATH, low_memory=False)

print("📌 Lendo dim_localidade:", DIM_LOC_PATH)
dim_loc = pd.read_csv(DIM_LOC_PATH, low_memory=False)

TOTAL = len(df)
checks = []
issues_samples = []

# colunas mínimas (ajuste conforme metadados quando você encontrar)
REQUIRED_COLS_MIN = ["contract_id","bank","ano_mes","product_id","financed_amount","outstanding_balance"]

# (1) Colunas obrigatórias
missing = [c for c in REQUIRED_COLS_MIN if c not in df.columns]
if missing:
    add_check(checks, "required_columns_min", "FAIL", len(missing), len(REQUIRED_COLS_MIN),
              len(missing)/len(REQUIRED_COLS_MIN)*100, f"Faltando: {missing}")
else:
    add_check(checks, "required_columns_min", "PASS", 0, len(REQUIRED_COLS_MIN), 0.0, "OK")

# (2) ano_mes em YYYYMM (6 dígitos) + mês válido
if "ano_mes" in df.columns:
    s = df["ano_mes"].astype(str).str.strip()

    # aceita "202501", "202501.0" (caso venha float) -> normaliza
    s = s.str.replace(r"\.0$", "", regex=True)

    mask_digits6 = s.str.match(r"^\d{6}$", na=False)
    year = pd.to_numeric(s.str.slice(0,4), errors="coerce")
    month = pd.to_numeric(s.str.slice(4,6), errors="coerce")
    mask_month = month.between(1,12, inclusive="both")
    # faixa de ano (ajustável)
    mask_year = year.between(1990, 2100, inclusive="both")

    mask_ok = mask_digits6 & mask_month & mask_year
    bad = int((~mask_ok).sum())
    pct = bad/TOTAL*100 if TOTAL else 0
    add_check(checks, "ano_mes_format_YYYYMM", "PASS" if bad==0 else "FAIL", bad, TOTAL, pct,
              "Esperado: YYYYMM (6 dígitos), mês 01-12.")
    if bad > 0:
        issues_samples.append(df.loc[~mask_ok, ["ano_mes"]].head(50).assign(issue="ano_mes_invalid_format"))
else:
    add_check(checks, "ano_mes_format_YYYYMM", "FAIL", details="Coluna ano_mes ausente.")

# (3) Nulos em colunas mínimas
for c in REQUIRED_COLS_MIN:
    if c not in df.columns:
        continue
    n = int(df[c].isna().sum())
    if n > 0:
        add_check(checks, f"{c}_nan_required", "FAIL", n, TOTAL, n/TOTAL*100 if TOTAL else 0, "Coluna mínima com NaN.")
        issues_samples.append(df.loc[df[c].isna(), ["contract_id","ano_mes",c]].head(50).assign(issue=f"{c}_nan"))
    else:
        add_check(checks, f"{c}_nan", "PASS", 0, TOTAL, 0.0, "OK")

# (4) Numéricos (cast e não-negativos)
for col in ["financed_amount","outstanding_balance"]:
    if col not in df.columns:
        continue
    x = pd.to_numeric(df[col], errors="coerce")
    bad_cast = int(x.isna().sum())
    add_check(checks, f"{col}_numeric_castable", "PASS" if bad_cast==0 else "WARN",
              bad_cast, TOTAL, bad_cast/TOTAL*100 if TOTAL else 0, "errors=coerce")
    neg = int((x.dropna() < 0).sum())
    add_check(checks, f"{col}_non_negative", "PASS" if neg==0 else "FAIL",
              neg, TOTAL, neg/TOTAL*100 if TOTAL else 0, "Esperado >= 0")

# (5) FK produto
if "product_id" in df.columns and "product_id" in dim_prod.columns:
    df_pid = to_int64(df["product_id"])
    dim_pid = to_int64(dim_prod["product_id"]).dropna().unique()
    mask_has = df_pid.notna()
    mask_invalid = mask_has & (~df_pid.isin(dim_pid))
    bad = int(mask_invalid.sum())
    add_check(checks, "fk_product_id_dim_produto", "PASS" if bad==0 else "FAIL",
              bad, TOTAL, bad/TOTAL*100 if TOTAL else 0, "product_id deve existir na dim_produto")
    if bad > 0:
        issues_samples.append(df.loc[mask_invalid, ["contract_id","ano_mes","product_id"]].head(50).assign(issue="fk_product_id_broken"))
else:
    add_check(checks, "fk_product_id_dim_produto", "WARN", details="Não foi possível validar FK.")

# (6) FK localidade
if "location_id" in df.columns and "location_id" in dim_loc.columns:
    df_lid = to_int64(df["location_id"])
    dim_lid = to_int64(dim_loc["location_id"]).dropna().unique()
    mask_has = df_lid.notna()
    mask_invalid = mask_has & (~df_lid.isin(dim_lid))
    bad = int(mask_invalid.sum())
    add_check(checks, "fk_location_id_dim_localidade", "PASS" if bad==0 else "FAIL",
              bad, TOTAL, bad/TOTAL*100 if TOTAL else 0, "location_id deve existir na dim_localidade")
    if bad > 0:
        issues_samples.append(df.loc[mask_invalid, ["contract_id","ano_mes","location_id"]].head(50).assign(issue="fk_location_id_broken"))
else:
    add_check(checks, "fk_location_id_dim_localidade", "WARN", details="Não foi possível validar FK.")

# (7) risk_score em [0,1] se existir
if "risk_score" in df.columns:
    x = pd.to_numeric(df["risk_score"], errors="coerce")
    mask_bad = x.notna() & ((x < 0) | (x > 1))
    bad = int(mask_bad.sum())
    add_check(checks, "risk_score_range_0_1", "PASS" if bad==0 else "FAIL",
              bad, TOTAL, bad/TOTAL*100 if TOTAL else 0, "Esperado entre 0 e 1.")
    if bad > 0:
        issues_samples.append(df.loc[mask_bad, ["contract_id","ano_mes","risk_score"]].head(50).assign(issue="risk_score_out_of_range"))

# (8) Duplicidade soft (contract_id, ano_mes, bank)
key_cols = [c for c in ["contract_id","ano_mes","bank"] if c in df.columns]
if len(key_cols) == 3:
    dup = df.duplicated(subset=key_cols, keep=False)
    bad = int(dup.sum())
    add_check(checks, "duplicate_rows_on_key_soft", "PASS" if bad==0 else "WARN",
              bad, TOTAL, bad/TOTAL*100 if TOTAL else 0, f"Duplicidade na chave soft: {key_cols}")
    if bad > 0:
        issues_samples.append(df.loc[dup, key_cols].head(50).assign(issue="duplicate_soft_key"))

# salvar
summary = pd.DataFrame(checks).sort_values(["status","check"])
display(summary)

LOG_DIR.mkdir(parents=True, exist_ok=True)
summary.to_csv(OUT_SUMMARY, index=False)

if issues_samples:
    issues = pd.concat(issues_samples, ignore_index=True)
    issues.to_csv(OUT_ISSUES, index=False)
    print("✅ Issues sample salvo em:", OUT_ISSUES)
else:
    print("✅ Nenhuma issue amostrada.")

print("✅ Summary salvo em:", OUT_SUMMARY)


📌 Lendo fato: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_clean_v4.csv
📌 Lendo dim_produto: /content/drive/MyDrive/Case/case-shield-hidra/data/raw/dim_produto.csv
📌 Lendo dim_localidade: /content/drive/MyDrive/Case/case-shield-hidra/data/raw/dim_localidade.csv


,check,status,bad_count,total_count,bad_pct,details
1,ano_mes_format_YYYYMM,PASS,0,5975,0.0,"Esperado: YYYYMM (6 dígitos), mês 01-12."
4,ano_mes_nan,PASS,0,5975,0.0,OK
3,bank_nan,PASS,0,5975,0.0,OK
2,contract_id_nan,PASS,0,5975,0.0,OK
15,duplicate_rows_on_key_soft,PASS,0,5975,0.0,"Duplicidade na chave soft: ['contract_id', 'an..."
6,financed_amount_nan,PASS,0,5975,0.0,OK
9,financed_amount_non_negative,PASS,0,5975,0.0,Esperado >= 0
8,financed_amount_numeric_castable,PASS,0,5975,0.0,errors=coerce
13,fk_location_id_dim_localidade,PASS,0,5975,0.0,location_id deve existir na dim_localidade
12,fk_product_id_dim_produto,PASS,0,5975,0.0,product_id deve existir na dim_produto


✅ Nenhuma issue amostrada.
✅ Summary salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/validation_step11_summary_v1.csv


## Passo 12 — Auditoria comparativa (Antes vs Depois) e geração da base final GOLD

O objetivo deste passo é consolidar a rastreabilidade das correções aplicadas ao longo do pipeline, comparando a base **antes** das correções com a base **após** as correções finais (versão validada), quantificando:

1. **Linhas removidas** (volume e percentual);
2. **Chaves removidas/adicionadas** (impacto por integridade e regras de limpeza);
3. **Linhas modificadas** (quantas linhas sofreram algum ajuste em pelo menos um campo);
4. **Células modificadas** (quantidade de valores alterados e percentual sobre o total comparado);
5. Exportação da versão final **GOLD** para consumo em banco de dados e BI.

### 12.1 Critérios de comparação
- A comparação é feita por uma **chave candidata (soft key)**: `contract_id`, `bank`, `ano_mes`.
- A auditoria separa:
  - **remoções**: registros que existiam no “antes” e não existem no “depois”;
  - **modificações**: registros presentes em ambas as versões, mas com diferenças em um ou mais campos.
- Campos numéricos (ex.: `financed_amount`, `outstanding_balance`, `risk_score`) são comparados de forma robusta (com tolerância para ponto flutuante).
- Colunas técnicas geradas apenas para suporte (ex.: versões `_int` ou flags) podem ser excluídas do comparativo e/ou removidas do GOLD.


### 12.2 Evidências e rastreabilidade (logs)
Foram gerados arquivos de auditoria para suportar a reprodutibilidade e o detalhamento das alterações:

- **Resumo da comparação (métricas e mudanças por coluna):**  
  `quality_logs/step12_comparison_summary.csv`

- **Amostra de registros alterados (valores antes/depois):**  
  `quality_logs/step12_changes_sample.csv`

### 12.3 Geração da base final GOLD
A versão final **GOLD** foi exportada a partir do dataset validado, removendo colunas técnicas e garantindo:

- chaves e campos essenciais presentes;
- integridade referencial preservada;
- ausência de valores inválidos detectados nas validações finais;
- estrutura pronta para carga em banco e consumo no BI.

**Arquivo gerado:**  
`data/processed/fato_contratos_gold.csv`

**Conclusão:** o Passo 12 encerra o processo com rastreabilidade completa das mudanças (remoções e correções), além de disponibilizar a versão GOLD final para uso produtivo.


In [14]:
# =====================================================================================
# PASSO 12 — Auditoria comparativa (Antes vs Depois) e geração da base final GOLD
# =====================================================================================

# =========================
# CONFIG
# =========================
processed_dir = "/content/drive/MyDrive/Case/case-shield-hidra/data/processed"
log_dir      = "/content/drive/MyDrive/Case/case-shield-hidra/quality_logs"

pattern = os.path.join(processed_dir, "fato_contratos_clean_v*.csv")

# Colunas técnicas que você pode remover do GOLD e ignorar na comparação
DROP_TECH_COLS = [
    "product_id_int", "product_id_num",
    "financed_amount_imputed", "financed_amount_imputed_flag",
    "risk_score_normalizado", "risk_score_norm"
]

# Chave de comparação (soft key)
KEY = ["contract_id", "bank", "ano_mes"]

# Dica de colunas numéricas
NUM_COLS_HINT = ["financed_amount", "outstanding_balance", "risk_score"]

# tolerância p/ float
FLOAT_TOL = 1e-6

# =========================
# FUNÇÕES
# =========================
def read_csv_smart(path: str) -> pd.DataFrame:
    try:
        return pd.read_csv(path, low_memory=False)
    except Exception:
        return pd.read_csv(path, low_memory=False, sep=";")

def safe_strip(x):
    if pd.isna(x):
        return np.nan
    return str(x).strip()

def extract_version(fp: str) -> int:
    m = re.search(r"_v(\d+)\.csv$", fp)
    return int(m.group(1)) if m else -1

# =========================
# 1) DESCOBRE BEFORE/AFTER
# =========================
files = sorted(glob.glob(pattern), key=extract_version)

if len(files) < 1:
    raise FileNotFoundError(
        f"Não encontrei pelo menos 1 versão em: {pattern}\n"
        f"Arquivos encontrados: {files}"
    )

before_path = "/content/drive/MyDrive/Case/case-shield-hidra/data/raw/fato_contratos.csv"  # raw
after_path  = files[-1]  # última

print("✅ Arquivos encontrados (ordem por versão):")
for f in files:
    print(" -", os.path.basename(f))

print("\n📌 BEFORE (primeira versão):", before_path)
print("📌 AFTER  (última versão)  :", after_path)

# =========================
# 2) LOAD
# =========================
df_before = read_csv_smart(before_path)
df_after  = read_csv_smart(after_path)

# =========================
# 3) MÉTRICAS DE LINHAS
# =========================
rows_before = len(df_before)
rows_after  = len(df_after)
rows_removed = rows_before - rows_after
rows_removed_pct = (rows_removed / rows_before * 100) if rows_before else 0.0

# =========================
# 4) NORMALIZA CHAVE
# =========================
for col in KEY:
    if col in df_before.columns:
        df_before[col] = df_before[col].map(safe_strip)
    if col in df_after.columns:
        df_after[col] = df_after[col].map(safe_strip)

df_before = df_before.drop_duplicates(subset=KEY, keep="first")
df_after  = df_after.drop_duplicates(subset=KEY, keep="first")

keys_before = set(map(tuple, df_before[KEY].values))
keys_after  = set(map(tuple, df_after[KEY].values))

keys_removed = keys_before - keys_after
keys_added   = keys_after - keys_before
keys_common  = keys_before & keys_after

removed_keys_count = len(keys_removed)
added_keys_count   = len(keys_added)
common_keys_count  = len(keys_common)
removed_keys_pct = (removed_keys_count / len(keys_before) * 100) if len(keys_before) else 0.0

# =========================
# 5) COMPARAÇÃO DE VALORES (somente chaves comuns)
# =========================
common_cols = [c for c in df_before.columns if c in df_after.columns and c not in KEY]
common_cols = [c for c in common_cols if c not in DROP_TECH_COLS]  # ignora cols técnicas

b = df_before.set_index(KEY)
a = df_after.set_index(KEY)

b_common = b.loc[list(keys_common), common_cols].copy()
a_common = a.loc[list(keys_common), common_cols].copy()

num_cols = [c for c in NUM_COLS_HINT if c in common_cols]
txt_cols = [c for c in common_cols if c not in num_cols]

changes_per_col = {}
changed_rows_mask = pd.Series(False, index=b_common.index)

# numéricas com tolerância
for c in num_cols:
    b_vals = pd.to_numeric(b_common[c], errors="coerce")
    a_vals = pd.to_numeric(a_common[c], errors="coerce")
    eq = (b_vals.isna() & a_vals.isna()) | (np.abs(b_vals - a_vals) <= FLOAT_TOL)
    diff = ~eq
    changes_per_col[c] = int(diff.sum())
    changed_rows_mask |= diff

# texto/ids
for c in txt_cols:
    b_vals = b_common[c].map(safe_strip)
    a_vals = a_common[c].map(safe_strip)
    eq = (b_vals.isna() & a_vals.isna()) | (b_vals == a_vals)
    diff = ~eq
    changes_per_col[c] = int(diff.sum())
    changed_rows_mask |= diff

changed_rows_count = int(changed_rows_mask.sum())
changed_rows_pct = (changed_rows_count / len(b_common) * 100) if len(b_common) else 0.0

total_cells_compared = len(b_common) * len(common_cols)
modified_cells_total = int(sum(changes_per_col.values()))
modified_cells_pct = (modified_cells_total / total_cells_compared * 100) if total_cells_compared else 0.0

# =========================
# 6) SALVA LOGS
# =========================
os.makedirs(log_dir, exist_ok=True)

summary_path = os.path.join(log_dir, "step12_comparison_summary.csv")
changes_sample_path = os.path.join(log_dir, "step12_changes_sample.csv")

summary_rows = [
    {"metric":"before_path", "value": before_path},
    {"metric":"after_path", "value": after_path},
    {"metric":"rows_before", "value": rows_before},
    {"metric":"rows_after", "value": rows_after},
    {"metric":"rows_removed", "value": rows_removed},
    {"metric":"rows_removed_pct", "value": round(rows_removed_pct, 6)},
    {"metric":"unique_keys_before", "value": len(keys_before)},
    {"metric":"unique_keys_after", "value": len(keys_after)},
    {"metric":"keys_removed_count", "value": removed_keys_count},
    {"metric":"keys_removed_pct", "value": round(removed_keys_pct, 6)},
    {"metric":"keys_added_count", "value": added_keys_count},
    {"metric":"keys_common_count", "value": common_keys_count},
    {"metric":"changed_rows_common_keys_count", "value": changed_rows_count},
    {"metric":"changed_rows_common_keys_pct", "value": round(changed_rows_pct, 6)},
    {"metric":"modified_cells_total", "value": modified_cells_total},
    {"metric":"modified_cells_pct", "value": round(modified_cells_pct, 6)},
    {"metric":"columns_compared_count", "value": len(common_cols)},
    {"metric":"columns_compared_list", "value": ", ".join(common_cols)},
]

for c in sorted(changes_per_col.keys()):
    summary_rows.append({"metric": f"changed_cells__{c}", "value": changes_per_col[c]})

pd.DataFrame(summary_rows).to_csv(summary_path, index=False)

# sample de alterações (até 200)
changed_idx = b_common.index[changed_rows_mask].tolist()[:200]
sample_before = b_common.loc[changed_idx].copy()
sample_after  = a_common.loc[changed_idx].copy()

sample_out = pd.DataFrame(index=sample_before.index)
for c in common_cols:
    sample_out[f"{c}__before"] = sample_before[c].values
    sample_out[f"{c}__after"]  = sample_after[c].values

sample_out = sample_out.reset_index()
sample_out.to_csv(changes_sample_path, index=False)

# =========================
# 7) GERA GOLD (a partir do AFTER)
# =========================
gold_path = os.path.join(processed_dir, "fato_contratos_gold.csv")

df_gold = df_after.copy()
cols_to_drop = [c for c in DROP_TECH_COLS if c in df_gold.columns]
df_gold = df_gold.drop(columns=cols_to_drop, errors="ignore")

ordered = [c for c in KEY if c in df_gold.columns] + [c for c in df_gold.columns if c not in KEY]
df_gold = df_gold[ordered]

df_gold.to_csv(gold_path, index=False)

# =========================
# 8) PRINT FINAL + DOWNLOAD
# =========================
print("\n✅ PASSO 12 — COMPARAÇÃO ANTES vs DEPOIS")
print(f"✅ Linhas antes: {rows_before}")
print(f"✅ Linhas depois: {rows_after}")
print(f"✅ Linhas removidas: {rows_removed} ({rows_removed_pct:.6f}%)")

print(f"\n✅ Chaves removidas: {removed_keys_count} ({removed_keys_pct:.6f}%)")
print(f"✅ Chaves adicionadas: {added_keys_count}")
print(f"✅ Chaves comuns: {common_keys_count}")

print(f"\n✅ Linhas (chaves comuns) com ≥1 alteração: {changed_rows_count} ({changed_rows_pct:.6f}%)")
print(f"✅ Células modificadas (total): {modified_cells_total} ({modified_cells_pct:.6f}%)")

print(f"\n✅ Summary salvo em: {summary_path}")
print(f"✅ Sample de mudanças salvo em: {changes_sample_path}")
print(f"✅ GOLD salvo em: {gold_path}")

try:
    from google.colab import files
    files.download(gold_path)
except Exception as e:
    print("\n⚠️ Não consegui disparar o download automaticamente.")
    print("Baixe pelo painel de arquivos do Colab.")
    print("Erro:", e)


✅ Arquivos encontrados (ordem por versão):
 - fato_contratos_clean_v1.csv
 - fato_contratos_clean_v2.csv
 - fato_contratos_clean_v3.csv
 - fato_contratos_clean_v4.csv

📌 BEFORE (primeira versão): /content/drive/MyDrive/Case/case-shield-hidra/data/raw/fato_contratos.csv
📌 AFTER  (última versão)  : /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fato_contratos_clean_v4.csv

✅ PASSO 12 — COMPARAÇÃO ANTES vs DEPOIS
✅ Linhas antes: 6000
✅ Linhas depois: 5975
✅ Linhas removidas: 25 (0.416667%)

✅ Chaves removidas: 66 (1.102941%)
✅ Chaves adicionadas: 57
✅ Chaves comuns: 5918

✅ Linhas (chaves comuns) com ≥1 alteração: 30 (0.506928%)
✅ Células modificadas (total): 30 (0.063366%)

✅ Summary salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/step12_comparison_summary.csv
✅ Sample de mudanças salvo em: /content/drive/MyDrive/Case/case-shield-hidra/quality_logs/step12_changes_sample.csv
✅ GOLD salvo em: /content/drive/MyDrive/Case/case-shield-hidra/data/processed/fa

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>